In [ ]:
# "Dmoz_Computers","Dmoz_Health","Dmoz_Science","Dmoz_Sports"
# "classic4","NSF","re8","review_polarity", "Industry_Sector"
# "SyskillWebert","webkb_parsed", "CSTR"
dataset_name = "Dmoz_Computers"

# Imports, load data , training functions and database creation

## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ipython-autotime
%load_ext autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.6 MB/s eta 0:00:00
time: 558 µs (started: 2024-12-26 17:19:50 +00:00)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os, errno
import networkx as nx
from networkx.algorithms import bipartite
import json
import yaml
import ast
from sklearn.metrics import classification_report
from sklearn.neighbors import NearestNeighbors
import pickle
import time

import tensorflow as tf

time: 11.3 s (started: 2024-12-26 17:19:50 +00:00)


In [ ]:
import torch

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.5.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 39.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
time: 35.8 s (started: 2024-12-26 17:20:01 +00:00)


In [ ]:
import torch_geometric
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.utils.convert import from_networkx
from torch_geometric.nn import SAGEConv, GATConv, Linear, to_hetero

time: 12.7 s (started: 2024-12-26 17:20:37 +00:00)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

time: 3.69 ms (started: 2024-12-26 17:20:50 +00:00)


## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

time: 31.1 s (started: 2024-12-26 17:20:50 +00:00)


In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,Artificial_Intelligence_1434272.txt,Principal Curves Page Introduction to principa...,Artificial_Intelligence,"[(principal curves, 0.7236), (curves summary, ...","[(introduction principal curves, 0.8212), (pri...","[(introduction principal curves, 0.8212), (pri...","[-0.034250993, -0.054697827, 0.02771881, -0.00...","[[-0.035950914, -0.084732965, -0.01326547, -0....","[[-0.068310104, -0.07682108, -0.0070457095, -0...","[[-0.068310104, -0.07682108, -0.0070457095, -0...","[[-0.034250993, -0.054697827, 0.02771881, -0.0..."
1,1,Artificial_Intelligence_1433615.txt,Iowa State University - Artificial Intelligenc...,Artificial_Intelligence,"[(university artificial, 0.5531), (intelligenc...","[(university artificial intelligence, 0.6874),...","[(university artificial intelligence, 0.6874),...","[-0.037128627, -0.054665048, -0.023520134, -0....","[[0.013373093, -0.042429943, 0.0018845485, -0....","[[-0.03555484, -0.020074962, 0.039645746, -0.0...","[[-0.03555484, -0.020074962, 0.039645746, -0.0...","[[-0.037128627, -0.054665048, -0.023520134, -0..."
2,2,Artificial_Intelligence_1434400.txt,"Paccanaro, Alberto Learning distributed repres...",Artificial_Intelligence,"[(concepts relational, 0.5298), (distributed r...","[(distributed representation concepts, 0.5994)...","[(distributed representation concepts, 0.5994)...","[0.017933754, -0.093935154, -0.07233604, 0.049...","[[0.09489803, -0.010703894, -0.07612236, 0.007...","[[-0.055929314, -0.041734546, -0.06326631, -0....","[[-0.055929314, -0.041734546, -0.06326631, -0....","[[0.017933754, -0.093935154, -0.07233604, 0.04..."
3,3,Artificial_Intelligence_1434281.txt,Attrasoft Provider of a number of neural netwo...,Artificial_Intelligence,"[(sound recognition, 0.5352), (attrasoft provi...","[(image sound recognition, 0.5388), (neural ne...","[(image sound recognition, 0.5388), (sound rec...","[-0.09700832, -0.123844825, -0.025894646, 0.02...","[[-0.043651912, -0.09779582, -0.024362028, -0....","[[-0.02410375, -0.06722981, -0.016101753, -0.0...","[[-0.02410375, -0.06722981, -0.016101753, -0.0...","[[-0.09700832, -0.123844825, -0.025894646, 0.0..."
4,4,Artificial_Intelligence_1434141.txt,LibML A machine learning library. New implemen...,Artificial_Intelligence,"[(machine learning, 0.614), (libml machine, 0....","[(libml machine learning, 0.8631), (machine le...","[(libml machine learning, 0.8631), (machine le...","[-0.08543352, -0.08932488, -0.019404357, -0.04...","[[-0.024391863, 0.0032444268, 0.054267664, -0....","[[-0.06194022, -0.10829034, 0.034314997, -0.02...","[[-0.06194022, -0.10829034, 0.034314997, -0.02...","[[-0.064178735, -0.12423259, -0.01962825, -0.0..."


time: 150 ms (started: 2024-12-26 17:21:21 +00:00)


## Training functions and Model Definitions

In [ ]:
def train(model,optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['document'].train_mask
    loss = F.cross_entropy(out['document'][mask], data['document'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)

time: 910 µs (started: 2024-12-26 17:21:21 +00:00)


In [ ]:
def test(model):
    model.eval()
    out = model(data.x_dict, data.edge_index_dict)
    pred = out['document'].argmax(dim=1)
    # print("PREDICTIONS ->", pred)
    accs = []
    for mask in [data['document'].train_mask, data['document'].test_mask]:
        accs.append(int((pred[mask] == data['document'].y[mask]).sum()) / int(mask.sum()))
    return accs

time: 836 µs (started: 2024-12-26 17:21:21 +00:00)


In [ ]:
def get_y(model):
    model.eval()
    out = model(data.x_dict, data.edge_index_dict)
    pred = out['document'].argmax(dim=1)
    print("PREDICTIONS ->", pred)

    # y_pred, y_true
    return pred[data['document'].test_mask],data['document'].y[data['document'].test_mask]

time: 357 µs (started: 2024-12-26 17:21:21 +00:00)


### GNN Model

In [ ]:
# First Network
from torch_geometric.nn import GraphConv

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GraphConv(-1, hidden_channels)
        self.conv2 = GraphConv(-1, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

time: 483 µs (started: 2024-12-26 17:21:21 +00:00)


### GAT Model

In [ ]:
# Second Network
from torch_geometric.nn import GATv2Conv

class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GATv2Conv((-1, -1), hidden_channels, add_self_loops=False)
        self.lin1 = Linear(-1, hidden_channels)
        self.conv2 = GATv2Conv((-1, -1), out_channels, add_self_loops=False)
        self.lin2 = Linear(-1, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) + self.lin1(x)
        x = x.relu()
        x = self.conv2(x, edge_index) + self.lin2(x)
        return x

time: 556 µs (started: 2024-12-26 17:21:21 +00:00)


### iteration

In [ ]:
def iteration(i,GCN_or_GAT,rotulated):
    # for i in range(10):
    print(f"\n===============================================")
    print(f"=================== MODEL {i} ===================")
    print(f"===============================================\n")

    # Defining masks
    with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks/mask_rot'+str(rotulated)+'_'+str(i)+'.pkl', 'rb') as f:
        aux = pickle.load(f)
        train_mask, val_mask, test_mask = aux[0],aux[1],aux[2]

    data['document'].train_mask = train_mask
    data['document'].val_mask = val_mask
    data['document'].test_mask = test_mask

    # Generate MODEL and Optmizer
    if GCN_or_GAT == "GCN":
        model = GNN(hidden_channels=64, out_channels=class_number)
    elif GCN_or_GAT =="GAT":
        model = GAT(hidden_channels=64, out_channels=class_number)

    model = to_hetero(model, data.metadata(), aggr='sum')
    model.to(device) # GPU

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    #contagem de tempo
    start = time.time()

    # Training
    L_loss = []
    for epoch in range(1, 1000):
        loss = train(model,optimizer)
        L_loss.append(loss)
        train_acc, test_acc = test(model)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Test: {test_acc:.4f}')
        df_temp = pd.DataFrame(L_loss)
        if len(df_temp >=5):
          loss_es =  pd.DataFrame(L_loss).tail(5)[0].std()
          print('Early stopping: ',loss_es)
          if loss_es <= 0.01:
            break

    #contagem de tempo
    end = time.time()

    # Creating Classification Report
    y_pred,y_true = get_y(model)
    y_pred = y_pred.cpu()
    y_true = y_true.cpu()
    target_names = df["class"].unique()

    class_dit = classification_report(y_true, y_pred, target_names=target_names, output_dict = True)

    print('\nClassification Report:\n')
    print(classification_report(y_true, y_pred, target_names=target_names))

    #L_results.append([model_name,'GCN',rotulated,class_dit['macro avg']['f1-score']])
    # Updating Dataframes
    df_list[i][0].at[model_name+"_"+GCN_or_GAT, str(rotulated)] = class_dit['accuracy']               # acuracia
    df_list[i][1].at[model_name+"_"+GCN_or_GAT, str(rotulated)] = class_dit['macro avg']['precision'] # precision
    df_list[i][2].at[model_name+"_"+GCN_or_GAT, str(rotulated)] = class_dit['macro avg']['recall']    # recall
    df_list[i][3].at[model_name+"_"+GCN_or_GAT, str(rotulated)] = class_dit['macro avg']['f1-score']  # f1-score
    df_list[i][4].at[model_name+"_"+GCN_or_GAT, str(rotulated)] = class_dit['macro avg']['support']   # support
    df_list[i][5].at[model_name+"_"+GCN_or_GAT, str(rotulated)] = (end - start)                       # time

time: 3.19 ms (started: 2024-12-26 17:21:21 +00:00)


## Database

In [ ]:
# # Only execude if dafaframes do not exist YET

# df_list = []

# for i in range(10):
#     aux_list = []

#     aux_list.append(pd.DataFrame(columns = ['1','5','10','20','30','80%'])) # acuracia
#     aux_list.append(pd.DataFrame(columns = ['1','5','10','20','30','80%'])) # precision
#     aux_list.append(pd.DataFrame(columns = ['1','5','10','20','30','80%'])) # recall
#     aux_list.append(pd.DataFrame(columns = ['1','5','10','20','30','80%'])) # f1-score
#     aux_list.append(pd.DataFrame(columns = ['1','5','10','20','30','80%'])) # support
#     aux_list.append(pd.DataFrame(columns = ['1','5','10','20','30','80%'])) # time

#     df_list.append(aux_list)

#     try:
#         os.makedirs("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i))
#     except OSError as e:
#         if e.errno != errno.EEXIST:
#             raise

#     df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
#     df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
#     df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
#     df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
#     df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
#     df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 241 µs (started: 2024-12-26 17:21:21 +00:00)


In [ ]:
df_list = []
for i in range(10):
    aux_list = []
    aux_list.append(pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl")) # acuracia
    aux_list.append(pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl")) # precision
    aux_list.append(pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl")) # recall
    aux_list.append(pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl"))  # f1-score
    aux_list.append(pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl")) # support
    aux_list.append(pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl")) # time
    df_list.append(aux_list)

time: 35 s (started: 2024-12-26 17:21:21 +00:00)


In [ ]:
df_list[0][0]

,1,5,10,20,30
Dmoz_Computers_keyphrase2_GCN,0.180572,0.393195,0.446724,0.489474,0.51299
Dmoz_Computers_keyphrase2_GAT,0.263685,0.427007,0.479377,0.530044,0.540202
Dmoz_Computers_keyphrase3_GCN,0.235524,0.436151,0.481633,0.541338,0.54009
Dmoz_Computers_keyphrase3_GAT,0.244489,0.456034,0.499033,0.567544,0.569317
Dmoz_Computers_keyphrase23_GCN,0.200506,0.42488,0.495811,0.545285,0.556215
...,...,...,...,...,...
Industry_Sector_keyphrase2_GAT,0.10971,0.192075,0.264229,0.400956,0.272674
Industry_Sector_keyphrase3_GCN,0.126178,0.143885,0.246637,0.236446,0.170864
Industry_Sector_keyphrase3_GAT,0.108007,0.208176,0.338278,0.397575,0.410666
Industry_Sector_keyphrase23_GCN,0.117547,0.174832,0.22732,0.306168,0.370344


time: 21 ms (started: 2024-12-26 17:21:56 +00:00)


--------------------------------------
# Hetrogenous Graph for Keyphrase = 2
--------------------------------------

In [ ]:
print(df["class"].unique())
class_number = len(df["class"].unique())
print(class_number)

['Artificial_Intelligence' 'CAD_and_CAM' 'Companies' 'Computer_Science'
 'Consultants' 'Data_Formats' 'Data_Communications' 'Education' 'Graphics'
 'Hardware' 'Internet' 'Mobile_Computing' 'Multimedia' 'Open_Source'
 'Programming' 'Robotics' 'Security' 'Software' 'Systems']
19
time: 3.76 ms (started: 2024-12-26 17:21:56 +00:00)


In [ ]:
# Change here to change wich keypharse to use
keyphrase = "keyphrase2"

model_name = dataset_name+"_"+keyphrase

time: 320 µs (started: 2024-12-26 17:21:56 +00:00)


## Creating the Graph

### Defining Graph Nodes and Edges 👀

- `Nodes` - documents and contexts
- `Edges`
  - document <- has -> context
- `Labels` - documents classes


#### Nodes and Edges 👀

##### Defining Docmuent nodes, Context nodes and edges between them

In [ ]:
all_contexts_list =[]
edges1,edges2 = [],[]
cont = 0
new_edge_cont = 0

document_nodes = []
context_nodes = []

edges_tuple = []

sentences = []
cont_sentences = 0
dit_sentences = {}

# Creating graph nodes and edges
for i in range(len(df)):
    # adding document nodes for every doc in df
    document_nodes.append(df["text_embeddings"][i])

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list: # if NOT
            # add a new context node for every new keyphrase in df
            context_nodes.append(df[keyphrase+"_embeddings"][i][j])
            # add a new edge between doc and new context
            edges1.append(df["id"][i])
            edges2.append(new_edge_cont)
            edges_tuple.append((df["id"][i],new_edge_cont))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges1.append(df["id"][i])
            edges2.append(all_contexts_list.index(key[0]))
            edges_tuple.append((df["id"][i],all_contexts_list.index(key[0])))
            cont+=1

    # organize sentences, sentences_embeddings, and a dict with the corresponding document for each sentence
    aux = df['sentences_embeddings'][i]
    for j,key in enumerate(aux):
        sentences.append(key)
        dit_sentences[cont_sentences] = df["id"][i]
        cont_sentences += 1


document_nodes = np.array(document_nodes)
context_nodes = np.array(context_nodes)

time: 18.9 s (started: 2024-12-26 17:21:56 +00:00)


In [ ]:
dit_sentences[9500]

6538

time: 5.66 ms (started: 2024-12-26 17:23:36 +00:00)


In [ ]:
len(sentences)

14131

time: 9.46 ms (started: 2024-12-26 17:24:15 +00:00)


In [ ]:
cont_sentences

14131

time: 14.4 ms (started: 2024-12-26 17:40:11 +00:00)


In [ ]:
print("number of document nodes:",len(document_nodes))
print("number of context nodes:",len(context_nodes))
print("number of shared contexts:",cont)
print("number of direct edges (first dimension):",len(edges1))
print("number of direct edges (second dimension):",len(edges2))

number of document nodes: 9500
number of context nodes: 38955
number of shared contexts: 8231
number of direct edges (first dimension): 47186
number of direct edges (second dimension): 47186
time: 4.93 ms (started: 2024-08-17 14:57:47 +00:00)


In [ ]:
nbrs = NearestNeighbors(n_neighbors=15, metric='cosine').fit(context_nodes)

neighbors_list = nbrs.kneighbors(sentences, return_distance=False)

# cria aresta para cada vizinho encontrado
for i,neighbors in enumerate(neighbors_list):
        for n in neighbors:
            edges1.append(dit_sentences[i])
            edges2.append(n)
            edges_tuple.append((dit_sentences[i],n))

time: 13 s (started: 2024-08-17 14:57:47 +00:00)


##### Ajusting everything to Tensor Objects

In [ ]:
# transforms egdes to tensor
edges = np.array([edges1,edges2])
edges = torch.tensor(edges)

time: 49.4 ms (started: 2024-08-17 14:58:00 +00:00)


In [ ]:
# transforms nodes to tensor
document_nodes = np.array(document_nodes)
document_nodes = torch.tensor(document_nodes)
context_nodes = np.array(context_nodes)
context_nodes = torch.tensor(context_nodes)

time: 67.1 ms (started: 2024-08-17 14:58:00 +00:00)


In [ ]:
# Show documents nodes
document_nodes

tensor([[-0.0343, -0.0547,  0.0277,  ...,  0.0056, -0.0645,  0.0124],
        [-0.0371, -0.0547, -0.0235,  ...,  0.0408,  0.0109, -0.0873],
        [ 0.0179, -0.0939, -0.0723,  ..., -0.0052, -0.0241, -0.0033],
        ...,
        [-0.0668, -0.0605, -0.0190,  ...,  0.1015, -0.0920,  0.1007],
        [ 0.0360,  0.0188, -0.0016,  ...,  0.0011,  0.0394,  0.0837],
        [-0.0379,  0.0473, -0.0619,  ...,  0.0604,  0.0204, -0.0091]])

time: 83.2 ms (started: 2024-08-17 14:58:00 +00:00)


In [ ]:
# Quantity of document nodes
len(document_nodes)

9500

time: 2.83 ms (started: 2024-08-17 14:58:00 +00:00)


In [ ]:
# Show context nodes
context_nodes

tensor([[-0.0360, -0.0847, -0.0133,  ...,  0.0233,  0.0017, -0.0408],
        [-0.0184, -0.0183, -0.0022,  ...,  0.0476,  0.0608, -0.0448],
        [-0.0510, -0.0211, -0.0460,  ...,  0.0450,  0.0468, -0.0529],
        ...,
        [-0.0808,  0.0026, -0.0760,  ...,  0.0095,  0.0288,  0.0107],
        [-0.0247,  0.0546, -0.0195,  ...,  0.0589,  0.0089, -0.0067],
        [-0.0471,  0.0765, -0.0771,  ..., -0.0453, -0.0008, -0.0039]])

time: 5.05 ms (started: 2024-08-17 14:58:00 +00:00)


In [ ]:
# Quantity of context nodes
len(context_nodes)

38955

time: 2.66 ms (started: 2024-08-17 14:58:00 +00:00)


In [ ]:
# Quantity of edges
print(len(edges[0]))
print(len(edges[1]))

259151
259151
time: 623 µs (started: 2024-08-17 14:58:00 +00:00)


In [ ]:
# showing edges
print(edges)

tensor([[    0,     0,     0,  ...,  9499,  9499,  9499],
        [    0,     1,     2,  ..., 38953, 37323, 23098]])
time: 869 µs (started: 2024-08-17 14:58:00 +00:00)


#### Class Labels

In [ ]:
# All different classes
print(df["class"].unique())

['Artificial_Intelligence' 'CAD_and_CAM' 'Companies' 'Computer_Science'
 'Consultants' 'Data_Formats' 'Data_Communications' 'Education' 'Graphics'
 'Hardware' 'Internet' 'Mobile_Computing' 'Multimedia' 'Open_Source'
 'Programming' 'Robotics' 'Security' 'Software' 'Systems']
time: 2.57 ms (started: 2024-08-17 14:58:00 +00:00)


In [ ]:
# Creating labels for classification
# the dictionary is a numeric representation for each possible "document" class

dit = {}
for i,classe in enumerate(df["class"].unique()):
  dit[classe] = i

print(dit,'\n')

labels = df[["class"]]
for i in range(len(df[["class"]])):
    labels["class"][i] = dit[labels["class"][i]]
labels.head()

{'Artificial_Intelligence': 0, 'CAD_and_CAM': 1, 'Companies': 2, 'Computer_Science': 3, 'Consultants': 4, 'Data_Formats': 5, 'Data_Communications': 6, 'Education': 7, 'Graphics': 8, 'Hardware': 9, 'Internet': 10, 'Mobile_Computing': 11, 'Multimedia': 12, 'Open_Source': 13, 'Programming': 14, 'Robotics': 15, 'Security': 16, 'Software': 17, 'Systems': 18} 



<ipython-input-33-a018e46c97dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["class"][i] = dit[labels["class"][i]]


,class
0,0
1,0
2,0
3,0
4,0


time: 873 ms (started: 2024-08-17 14:58:00 +00:00)


In [ ]:
# tranfors class dataframe into tensor
y = labels["class"].tolist()
y = x_np = torch.tensor(y)
y

tensor([ 0,  0,  0,  ..., 18, 18, 18])

time: 4.97 ms (started: 2024-08-17 14:58:01 +00:00)


In [ ]:
len(y)

9500

time: 2.51 ms (started: 2024-08-17 14:58:01 +00:00)


### Testing Graph with Networkx




#### Defining overal graph in networkx maner

In [ ]:
# Run graph Representation for networkx
all_contexts_list =[]
edges_test = []
cont = 0
new_edge_cont = 0

document_nodes_test = []
context_nodes_test = []

# Creating graph nodes and edges
for i in range(len(df)):
    # adding new documents for every node
    document_nodes_test.append("doc_"+str(i)) # in the actual graph nodes -> documents embeddings

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list:
            # add a new context node for every new keyphrase in df
            context_nodes_test.append("contx_"+str(new_edge_cont)) # in the actual graph nodes -> context embeddings

            # add a new edge between doc and new context
            edges_test.append(("doc_"+str(df["id"][i]),"contx_"+str(new_edge_cont)))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges_test.append(("doc_"+str(df["id"][i]),"contx_"+str(all_contexts_list.index(key[0]))))
            cont+=1

time: 14 s (started: 2024-08-17 14:58:01 +00:00)


In [ ]:
edges_test = [("doc_"+str(i[0]),"contx_"+str(i[1])) for i in edges_tuple]

time: 532 ms (started: 2024-08-17 14:58:15 +00:00)


In [ ]:
print("number of document nodes:",len(document_nodes_test))
print("number of context nodes:",len(context_nodes_test))
print("number of edges:",len(edges_test))

number of document nodes: 9500
number of context nodes: 38955
number of edges: 259151
time: 1.42 ms (started: 2024-08-17 14:58:16 +00:00)


In [ ]:
print(edges_test[0:100])

[('doc_0', 'contx_0'), ('doc_0', 'contx_1'), ('doc_0', 'contx_2'), ('doc_0', 'contx_3'), ('doc_0', 'contx_4'), ('doc_1', 'contx_5'), ('doc_1', 'contx_6'), ('doc_1', 'contx_7'), ('doc_1', 'contx_8'), ('doc_1', 'contx_9'), ('doc_2', 'contx_10'), ('doc_2', 'contx_11'), ('doc_2', 'contx_12'), ('doc_2', 'contx_13'), ('doc_2', 'contx_14'), ('doc_3', 'contx_15'), ('doc_3', 'contx_16'), ('doc_3', 'contx_17'), ('doc_3', 'contx_18'), ('doc_3', 'contx_19'), ('doc_4', 'contx_20'), ('doc_4', 'contx_21'), ('doc_4', 'contx_22'), ('doc_4', 'contx_23'), ('doc_4', 'contx_24'), ('doc_5', 'contx_25'), ('doc_5', 'contx_26'), ('doc_5', 'contx_27'), ('doc_5', 'contx_28'), ('doc_5', 'contx_29'), ('doc_6', 'contx_30'), ('doc_6', 'contx_31'), ('doc_6', 'contx_32'), ('doc_6', 'contx_33'), ('doc_6', 'contx_34'), ('doc_7', 'contx_35'), ('doc_7', 'contx_36'), ('doc_7', 'contx_37'), ('doc_7', 'contx_38'), ('doc_7', 'contx_39'), ('doc_8', 'contx_40'), ('doc_8', 'contx_41'), ('doc_8', 'contx_42'), ('doc_8', 'contx_43'

#### Test graph Conectivity with networkx

In [ ]:
# Define Bipartide graph
B = nx.Graph()
B.add_nodes_from(document_nodes_test, bipartite=0)
B.add_nodes_from(context_nodes_test, bipartite=1)
B.add_edges_from(edges_test)

time: 398 ms (started: 2024-08-17 14:58:16 +00:00)


In [ ]:
nx.is_connected(B)

True

time: 124 ms (started: 2024-08-17 14:58:16 +00:00)


In [ ]:
# Number of conected elements
# if == 1 -> all elements of the graph are conected
print(nx.number_connected_components(B))

1
time: 120 ms (started: 2024-08-17 14:58:16 +00:00)


In [ ]:
# size of a cluster
print(len(nx.node_connected_component(B,'doc_0')))

48455
time: 99.7 ms (started: 2024-08-17 14:58:16 +00:00)


### Creating Graph HeteroData Object

In [ ]:
# Defining nodes, edges and class labels
from torch_geometric.data import HeteroData

data = HeteroData()

# nodes
data['document'].x = document_nodes
data['concept'].x = context_nodes

# edges
data['document', 'has', 'concept'].edge_index = edges

#class labels
data['document'].y = y


time: 1.18 ms (started: 2024-08-17 14:58:17 +00:00)


In [ ]:
# Setting graph to undirected
data = T.ToUndirected()(data)

time: 2.34 ms (started: 2024-08-17 14:58:17 +00:00)


In [ ]:
# Remove duplicate edges
data = T.RemoveDuplicatedEdges()(data)

time: 67.3 ms (started: 2024-08-17 14:58:17 +00:00)


In [ ]:
# Ensure date in using gpu
data = data.to(device)

time: 182 ms (started: 2024-08-17 14:58:17 +00:00)


In [ ]:
data

HeteroData(
  document={
    x=[9500, 384],
    y=[9500],
  },
  concept={ x=[38955, 384] },
  (document, has, concept)={ edge_index=[2, 223512] },
  (concept, rev_has, document)={ edge_index=[2, 223512] }
)

time: 6.82 ms (started: 2024-08-17 14:58:17 +00:00)


## Creating all masks

### Rotulated = 1

In [ ]:
rotulated = 1

time: 377 µs (started: 2024-08-16 14:10:51 +00:00)


#### Creating mask Split using RandomNodeSplit (if split not done before)

In [ ]:
# Using RandomNodeSplit to create the training, validation and testing masks
# RandomNodeSplit performs a randomic separtion of the dataset and generates the masks for the Data object
# by selecting validation and tesing sizes, RandomNodeSplit

# RandomNodeSplit: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.transforms.RandomNodeSplit.html#torch_geometric.transforms.RandomNodeSplit

for i in range(10):
    transform = T.RandomNodeSplit(split = 'test_rest', num_train_per_class = rotulated, num_val=0, num_test=1000)

    data = transform(data)

    with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks/mask_rot'+str(rotulated)+'_'+str(i)+'.pkl', 'wb') as f:
        pickle.dump([data['document'].train_mask, data['document'].val_mask,data['document'].test_mask], f)

time: 3.86 s (started: 2024-08-16 14:10:51 +00:00)


### Rotulated = 5

In [ ]:
rotulated = 5

time: 430 µs (started: 2024-08-16 14:10:55 +00:00)


#### Creating mask Split using RandomNodeSplit (if split not done before)

In [ ]:
# Using RandomNodeSplit to create the training, validation and testing masks
# RandomNodeSplit performs a randomic separtion of the dataset and generates the masks for the Data object
# by selecting validation and tesing sizes, RandomNodeSplit

# RandomNodeSplit: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.transforms.RandomNodeSplit.html#torch_geometric.transforms.RandomNodeSplit

for i in range(10):
    transform = T.RandomNodeSplit(split = 'test_rest', num_train_per_class = rotulated, num_val=0, num_test=1000)

    data = transform(data)

    with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks/mask_rot'+str(rotulated)+'_'+str(i)+'.pkl', 'wb') as f:
        pickle.dump([data['document'].train_mask, data['document'].val_mask,data['document'].test_mask], f)

time: 4.17 s (started: 2024-08-16 14:10:55 +00:00)




### Rotulated = 10

In [ ]:
rotulated = 10

time: 593 µs (started: 2024-08-16 14:10:59 +00:00)


#### Creating mask Split using RandomNodeSplit (if split not done before)

In [ ]:
# Using RandomNodeSplit to create the training, validation and testing masks
# RandomNodeSplit performs a randomic separtion of the dataset and generates the masks for the Data object
# by selecting validation and tesing sizes, RandomNodeSplit

# RandomNodeSplit: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.transforms.RandomNodeSplit.html#torch_geometric.transforms.RandomNodeSplit

for i in range(10):
    transform = T.RandomNodeSplit(split = 'test_rest', num_train_per_class = rotulated, num_val=0, num_test=1000)

    data = transform(data)

    with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks/mask_rot'+str(rotulated)+'_'+str(i)+'.pkl', 'wb') as f:
        pickle.dump([data['document'].train_mask, data['document'].val_mask,data['document'].test_mask], f)

time: 3.94 s (started: 2024-08-16 14:10:59 +00:00)


### Rotulated = 20

In [ ]:
rotulated = 20

time: 520 µs (started: 2024-08-16 14:11:03 +00:00)


#### Creating mask Split using RandomNodeSplit (if split not done before)

In [ ]:
# Using RandomNodeSplit to create the training, validation and testing masks
# RandomNodeSplit performs a randomic separtion of the dataset and generates the masks for the Data object
# by selecting validation and tesing sizes, RandomNodeSplit

# RandomNodeSplit: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.transforms.RandomNodeSplit.html#torch_geometric.transforms.RandomNodeSplit

for i in range(10):
    transform = T.RandomNodeSplit(split = 'test_rest', num_train_per_class = rotulated, num_val=0, num_test=1000)

    data = transform(data)

    with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks/mask_rot'+str(rotulated)+'_'+str(i)+'.pkl', 'wb') as f:
        pickle.dump([data['document'].train_mask, data['document'].val_mask,data['document'].test_mask], f)

time: 4.02 s (started: 2024-08-16 14:11:03 +00:00)




### Rotulated = 30

In [ ]:
rotulated = 30

time: 319 µs (started: 2024-08-16 14:11:07 +00:00)


#### Creating mask Split using RandomNodeSplit (if split not done before)

In [ ]:
# Using RandomNodeSplit to create the training, validation and testing masks
# RandomNodeSplit performs a randomic separtion of the dataset and generates the masks for the Data object
# by selecting validation and tesing sizes, RandomNodeSplit

# RandomNodeSplit: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.transforms.RandomNodeSplit.html#torch_geometric.transforms.RandomNodeSplit

for i in range(10):
    transform = T.RandomNodeSplit(split = 'test_rest', num_train_per_class = rotulated, num_val=0, num_test=1000)

    data = transform(data)

    with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks/mask_rot'+str(rotulated)+'_'+str(i)+'.pkl', 'wb') as f:
        pickle.dump([data['document'].train_mask, data['document'].val_mask,data['document'].test_mask], f)

time: 4.17 s (started: 2024-08-16 14:11:07 +00:00)


### Rotulated = 80%

In [ ]:
rotulated = '80%'

time: 281 µs (started: 2024-08-16 14:11:11 +00:00)


#### Creating mask Split using RandomNodeSplit (if split not done before)

In [ ]:
# Using RandomNodeSplit to create the training, validation and testing masks
# RandomNodeSplit performs a randomic separtion of the dataset and generates the masks for the Data object
# by selecting validation and tesing sizes, RandomNodeSplit

# RandomNodeSplit: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.transforms.RandomNodeSplit.html#torch_geometric.transforms.RandomNodeSplit

def percentage(percent, whole):
    return (percent * whole) / 100.0

perc = int(percentage(20,len(df['id'])))

print("20 % "+str(len(df['id']))+' = '+str(perc))

for i in range(10):

    transform = T.RandomNodeSplit(split = 'train_rest', num_val=0, num_test=perc)

    data = transform(data)

    with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks/mask_rot'+str(rotulated)+'_'+str(i)+'.pkl', 'wb') as f:
        pickle.dump([data['document'].train_mask, data['document'].val_mask,data['document'].test_mask], f)

20 % 10524 = 2104
time: 4.34 s (started: 2024-08-16 14:11:11 +00:00)


## TRAINING

### Training rotulated base = 1 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',1)


=================== MODEL 0 ===================

Epoch: 001, Loss: 5.4004, Train: 0.7500, Test: 0.4371
Early stopping:  nan
Epoch: 002, Loss: 1.0565, Train: 0.9375, Test: 0.5059
Early stopping:  3.0715911755936705
Epoch: 003, Loss: 0.1183, Train: 1.0000, Test: 0.5642
Early stopping:  2.818088877542161
Epoch: 004, Loss: 0.0080, Train: 0.9375, Test: 0.5950
Early stopping:  2.5468750375848677
Epoch: 005, Loss: 0.0601, Train: 1.0000, Test: 0.5684
Early stopping:  2.316853496770235
Epoch: 006, Loss: 0.0002, Train: 1.0000, Test: 0.5540
Early stopping:  0.45408106572987206
Epoch: 007, Loss: 0.0001, Train: 1.0000, Test: 0.5358
Early stopping:  0.05171516836837455
Epoch: 008, Loss: 0.0014, Train: 1.0000, Test: 0.5174
Early stopping:  0.025999795842934426
Epoch: 009, Loss: 0.0003, Train: 1.0000, Test: 0.4929
Early stopping:  0.026662325958785243
Epoch: 010, Loss: 0.0001, Train: 1.0000, Test: 0.4715
Early stopping:  0.0005273933703170028
PREDICTIONS -> tensor([ 0,  0,  0,  ...,  0, 15,  0], devi

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 434 ms (started: 2024-08-16 14:11:21 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',1)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7853, Train: 0.9375, Test: 0.3421
Early stopping:  nan
Epoch: 002, Loss: 2.4027, Train: 0.9375, Test: 0.4913
Early stopping:  0.27057770981811835
Epoch: 003, Loss: 1.9648, Train: 1.0000, Test: 0.5223
Early stopping:  0.4106037110549708
Epoch: 004, Loss: 1.4785, Train: 1.0000, Test: 0.5317
Early stopping:  0.5634700178713508
Epoch: 005, Loss: 1.0030, Train: 1.0000, Test: 0.5498
Early stopping:  0.710515949745173
Epoch: 006, Loss: 0.6062, Train: 1.0000, Test: 0.5672
Early stopping:  0.7205575660204593
Epoch: 007, Loss: 0.3269, Train: 1.0000, Test: 0.5837
Early stopping:  0.6593910279309918
Epoch: 008, Loss: 0.1638, Train: 1.0000, Test: 0.5913
Early stopping:  0.5320258563654219
Epoch: 009, Loss: 0.0816, Train: 1.0000, Test: 0.5932
Early stopping:  0.3748715333221112
Epoch: 010, Loss: 0.0421, Train: 1.0000, Test: 0.5933
Early stopping:  0.23001054749166383
Epoch: 011, Loss: 0.0228, Train: 1.0000, Test: 0.5939
Early stop

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 421 ms (started: 2024-08-16 14:11:29 +00:00)


### Training rotulated base = 5 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',5)


=================== MODEL 0 ===================

Epoch: 001, Loss: 4.0398, Train: 0.7875, Test: 0.5812
Early stopping:  nan
Epoch: 002, Loss: 0.9149, Train: 0.9500, Test: 0.6975
Early stopping:  2.209632152531065
Epoch: 003, Loss: 0.2483, Train: 0.9625, Test: 0.7274
Early stopping:  2.024229972181255
Epoch: 004, Loss: 0.1319, Train: 0.9875, Test: 0.7199
Early stopping:  1.836735418721627
Epoch: 005, Loss: 0.0841, Train: 0.9875, Test: 0.7160
Early stopping:  1.6859697310888826
Epoch: 006, Loss: 0.0481, Train: 1.0000, Test: 0.7228
Early stopping:  0.35987780383354745
Epoch: 007, Loss: 0.0103, Train: 1.0000, Test: 0.7236
Early stopping:  0.09204473502296306
Epoch: 008, Loss: 0.0083, Train: 0.9875, Test: 0.7237
Early stopping:  0.052390527048580306
Epoch: 009, Loss: 0.0134, Train: 1.0000, Test: 0.7302
Early stopping:  0.0329768010601154
Epoch: 010, Loss: 0.0073, Train: 1.0000, Test: 0.7349
Early stopping:  0.017258433826258766
Epoch: 011, Loss: 0.0021, Train: 1.0000, Test: 0.7342
Early st

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 420 ms (started: 2024-08-16 14:11:34 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',5)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7875, Train: 0.7250, Test: 0.5150
Early stopping:  nan
Epoch: 002, Loss: 2.4887, Train: 0.9625, Test: 0.7444
Early stopping:  0.2112764196137143
Epoch: 003, Loss: 2.1323, Train: 0.9750, Test: 0.7563
Early stopping:  0.3280061939801322
Epoch: 004, Loss: 1.7213, Train: 0.9875, Test: 0.7613
Early stopping:  0.4601076253367723
Epoch: 005, Loss: 1.3070, Train: 0.9875, Test: 0.7703
Early stopping:  0.5908299743660604
Epoch: 006, Loss: 0.9299, Train: 0.9875, Test: 0.7807
Early stopping:  0.6236397428920462
Epoch: 007, Loss: 0.6188, Train: 0.9875, Test: 0.7827
Early stopping:  0.6046690904124602
Epoch: 008, Loss: 0.3888, Train: 1.0000, Test: 0.7814
Early stopping:  0.5333736719718948
Epoch: 009, Loss: 0.2372, Train: 1.0000, Test: 0.7738
Early stopping:  0.4297793452648574
Epoch: 010, Loss: 0.1443, Train: 1.0000, Test: 0.7673
Early stopping:  0.31636437341703944
Epoch: 011, Loss: 0.0889, Train: 1.0000, Test: 0.7640
Early stop

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 404 ms (started: 2024-08-16 14:11:43 +00:00)


### Training rotulated base = 10 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',10)


=================== MODEL 0 ===================

Epoch: 001, Loss: 4.7920, Train: 0.6250, Test: 0.5451
Early stopping:  nan
Epoch: 002, Loss: 2.5343, Train: 0.7375, Test: 0.6108
Early stopping:  1.5964754171223337
Epoch: 003, Loss: 1.2421, Train: 0.9062, Test: 0.7360
Early stopping:  1.7967398700262387
Epoch: 004, Loss: 0.4119, Train: 0.9187, Test: 0.7469
Early stopping:  1.909373999572375
Epoch: 005, Loss: 0.3919, Train: 0.9000, Test: 0.7214
Early stopping:  1.849624808943473
Epoch: 006, Loss: 0.3520, Train: 0.9437, Test: 0.7836
Early stopping:  0.941697142460063
Epoch: 007, Loss: 0.1570, Train: 0.9625, Test: 0.7837
Early stopping:  0.42102660103833894
Epoch: 008, Loss: 0.1567, Train: 0.9812, Test: 0.7750
Early stopping:  0.12696667045238932
Epoch: 009, Loss: 0.1341, Train: 0.9750, Test: 0.7619
Early stopping:  0.12313510703427856
Epoch: 010, Loss: 0.1027, Train: 0.9938, Test: 0.7532
Early stopping:  0.09839461089058668
Epoch: 011, Loss: 0.0548, Train: 1.0000, Test: 0.7495
Early stop

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 451 ms (started: 2024-08-16 14:11:50 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',10)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7737, Train: 0.6250, Test: 0.5635
Early stopping:  nan
Epoch: 002, Loss: 2.4945, Train: 0.8063, Test: 0.6664
Early stopping:  0.19739813674865306
Epoch: 003, Loss: 2.1585, Train: 0.8938, Test: 0.7205
Early stopping:  0.3080349188391454
Epoch: 004, Loss: 1.7799, Train: 0.9313, Test: 0.7493
Early stopping:  0.42926132631503666
Epoch: 005, Loss: 1.4028, Train: 0.9437, Test: 0.7617
Early stopping:  0.547535317911406
Epoch: 006, Loss: 1.0584, Train: 0.9500, Test: 0.7663
Early stopping:  0.5737655378564829
Epoch: 007, Loss: 0.7710, Train: 0.9625, Test: 0.7723
Early stopping:  0.5536564737413016
Epoch: 008, Loss: 0.5506, Train: 0.9688, Test: 0.7734
Early stopping:  0.49114231859929
Epoch: 009, Loss: 0.3934, Train: 0.9750, Test: 0.7746
Early stopping:  0.40380389794874166
Epoch: 010, Loss: 0.2834, Train: 0.9750, Test: 0.7752
Early stopping:  0.309871338128894
Epoch: 011, Loss: 0.2036, Train: 0.9938, Test: 0.7756
Early stoppi

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 419 ms (started: 2024-08-16 14:12:00 +00:00)


### Training rotulated base = 20 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',20)


=================== MODEL 0 ===================

Epoch: 001, Loss: 3.7326, Train: 0.6156, Test: 0.5724
Early stopping:  nan
Epoch: 002, Loss: 1.4317, Train: 0.7688, Test: 0.6531
Early stopping:  1.6269482629452863
Epoch: 003, Loss: 0.9967, Train: 0.8219, Test: 0.7488
Early stopping:  1.4701592605030074
Epoch: 004, Loss: 0.6394, Train: 0.8562, Test: 0.7496
Early stopping:  1.3931768761189545
Epoch: 005, Loss: 0.5610, Train: 0.8719, Test: 0.7308
Early stopping:  1.3096682531375576
Epoch: 006, Loss: 0.5034, Train: 0.9187, Test: 0.7628
Early stopping:  0.388963047345332
Epoch: 007, Loss: 0.3068, Train: 0.9219, Test: 0.7819
Early stopping:  0.2528485931318276
Epoch: 008, Loss: 0.2602, Train: 0.9375, Test: 0.7927
Early stopping:  0.16393435989873603
Epoch: 009, Loss: 0.2342, Train: 0.9375, Test: 0.7939
Early stopping:  0.1489263953374671
Epoch: 010, Loss: 0.1595, Train: 0.9563, Test: 0.7770
Early stopping:  0.1292349331581183
Epoch: 011, Loss: 0.1498, Train: 0.9625, Test: 0.7679
Early stopp

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 440 ms (started: 2024-08-16 14:12:08 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',20)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7886, Train: 0.6875, Test: 0.5631
Early stopping:  nan
Epoch: 002, Loss: 2.5296, Train: 0.8344, Test: 0.7245
Early stopping:  0.1831494673788781
Epoch: 003, Loss: 2.2169, Train: 0.8187, Test: 0.7222
Early stopping:  0.28630885094650904
Epoch: 004, Loss: 1.8589, Train: 0.8406, Test: 0.7378
Early stopping:  0.4014999036127426
Epoch: 005, Loss: 1.4996, Train: 0.8844, Test: 0.7595
Early stopping:  0.5148209682582936
Epoch: 006, Loss: 1.1702, Train: 0.8906, Test: 0.7826
Early stopping:  0.5434541823379269
Epoch: 007, Loss: 0.8917, Train: 0.8969, Test: 0.7921
Early stopping:  0.5286180013141202
Epoch: 008, Loss: 0.6766, Train: 0.9031, Test: 0.7968
Early stopping:  0.4721872481064351
Epoch: 009, Loss: 0.5228, Train: 0.9125, Test: 0.7981
Early stopping:  0.3908737690074459
Epoch: 010, Loss: 0.4135, Train: 0.9250, Test: 0.8009
Early stopping:  0.30239315906968045
Epoch: 011, Loss: 0.3341, Train: 0.9406, Test: 0.8037
Early sto

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 455 ms (started: 2024-08-16 14:12:20 +00:00)


### Training rotulated base = 30 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',30)


=================== MODEL 0 ===================

Epoch: 001, Loss: 5.2296, Train: 0.5104, Test: 0.4516
Early stopping:  nan
Epoch: 002, Loss: 2.6893, Train: 0.6583, Test: 0.6624
Early stopping:  1.796272889663015
Epoch: 003, Loss: 2.1617, Train: 0.7875, Test: 0.7386
Early stopping:  1.6403131160804936
Epoch: 004, Loss: 1.1101, Train: 0.8583, Test: 0.7609
Early stopping:  1.74913772023399
Epoch: 005, Loss: 0.6282, Train: 0.8521, Test: 0.7371
Early stopping:  1.7988697519550108
Epoch: 006, Loss: 0.8400, Train: 0.8667, Test: 0.7545
Early stopping:  0.8942760773448944
Epoch: 007, Loss: 0.6156, Train: 0.8854, Test: 0.7810
Early stopping:  0.6418596336448062
Epoch: 008, Loss: 0.4209, Train: 0.8750, Test: 0.7803
Early stopping:  0.2623426299756426
Epoch: 009, Loss: 0.4496, Train: 0.9021, Test: 0.7933
Early stopping:  0.16800215866113555
Epoch: 010, Loss: 0.4099, Train: 0.9146, Test: 0.8036
Early stopping:  0.18351469423803254
Epoch: 011, Loss: 0.3413, Train: 0.9250, Test: 0.8062
Early stoppi

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 403 ms (started: 2024-08-16 14:12:29 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',30)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7771, Train: 0.7375, Test: 0.5742
Early stopping:  nan
Epoch: 002, Loss: 2.5240, Train: 0.8333, Test: 0.7305
Early stopping:  0.17892095836147712
Epoch: 003, Loss: 2.2173, Train: 0.8479, Test: 0.7551
Early stopping:  0.2803310560762672
Epoch: 004, Loss: 1.8566, Train: 0.8625, Test: 0.7670
Early stopping:  0.39729504336522065
Epoch: 005, Loss: 1.4904, Train: 0.8771, Test: 0.7767
Early stopping:  0.5138214588327777
Epoch: 006, Loss: 1.1568, Train: 0.8812, Test: 0.7868
Early stopping:  0.5474924705230528
Epoch: 007, Loss: 0.8810, Train: 0.8729, Test: 0.7961
Early stopping:  0.5339850755254507
Epoch: 008, Loss: 0.6734, Train: 0.8771, Test: 0.7999
Early stopping:  0.4732411864176396
Epoch: 009, Loss: 0.5291, Train: 0.8833, Test: 0.8042
Early stopping:  0.3851156697297833
Epoch: 010, Loss: 0.4317, Train: 0.8917, Test: 0.8079
Early stopping:  0.29045954142230473
Epoch: 011, Loss: 0.3629, Train: 0.9062, Test: 0.8135
Early st

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 429 ms (started: 2024-08-16 14:12:42 +00:00)


### Training rotulated base = 80% ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN','80%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 5.4239, Train: 0.1653, Test: 0.1721
Early stopping:  nan
Epoch: 002, Loss: 12.5963, Train: 0.1701, Test: 0.1716
Early stopping:  5.0716402002967955
Epoch: 003, Loss: 12.6517, Train: 0.2808, Test: 0.2779
Early stopping:  4.157081515254105
Epoch: 004, Loss: 9.7003, Train: 0.3666, Test: 0.3700
Early stopping:  3.4043249504935438
Epoch: 005, Loss: 6.1940, Train: 0.3171, Test: 0.3063
Early stopping:  3.4252941513867863
Epoch: 006, Loss: 6.8460, Train: 0.3470, Test: 0.3395
Early stopping:  3.061305589056082
Epoch: 007, Loss: 5.9098, Train: 0.3689, Test: 0.3489
Early stopping:  2.8789463899110386
Epoch: 008, Loss: 4.6041, Train: 0.4345, Test: 0.4068
Early stopping:  1.8897503744647153
Epoch: 009, Loss: 3.3294, Train: 0.4780, Test: 0.4658
Early stopping:  1.4052968555629086
Epoch: 010, Loss: 2.5441, Train: 0.4861, Test: 0.4742
Early stopping:  1.7741348707177251
Epoch: 011, Loss: 2.1945, Train: 0.4786, Test: 0.4742
Early stopp

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 396 ms (started: 2024-08-17 14:17:42 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT','80%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.9516, Train: 0.3708, Test: 0.3642
Early stopping:  nan
Epoch: 002, Loss: 2.8218, Train: 0.5242, Test: 0.5100
Early stopping:  0.09182567602547291
Epoch: 003, Loss: 2.6574, Train: 0.5462, Test: 0.5168
Early stopping:  0.1474472939959434
Epoch: 004, Loss: 2.4685, Train: 0.5671, Test: 0.5389
Early stopping:  0.20904086312435768
Epoch: 005, Loss: 2.2699, Train: 0.5899, Test: 0.5705
Early stopping:  0.27232267987651354
Epoch: 006, Loss: 2.0711, Train: 0.6104, Test: 0.5905
Early stopping:  0.29885855196152794
Epoch: 007, Loss: 1.8796, Train: 0.6309, Test: 0.6184
Early stopping:  0.3088029222987719
Epoch: 008, Loss: 1.7019, Train: 0.6404, Test: 0.6263
Early stopping:  0.30419721714024717
Epoch: 009, Loss: 1.5475, Train: 0.6449, Test: 0.6311
Early stopping:  0.2871609024585015
Epoch: 010, Loss: 1.4215, Train: 0.6512, Test: 0.6321
Early stopping:  0.25878232986934796
Epoch: 011, Loss: 1.3216, Train: 0.6562, Test: 0.6411
Early

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 515 ms (started: 2024-08-17 14:18:22 +00:00)


--------------------------------------
# Hetrogenous Graph for Keyphrase = 3
--------------------------------------

In [ ]:
print(df["class"].unique())
class_number = len(df["class"].unique())
print(class_number)

['data_management' 'ecology' 'economic' 'geophysics' 'gravitional_theory'
 'hydro' 'math' 'metals' 'networking' 'neuroscience' 'oceanography'
 'politic' 'sociology' 'software_engineering' 'statistics'
 'theory_computing']
16
time: 5.15 ms (started: 2024-08-16 14:12:42 +00:00)


In [ ]:
# Change here to change wich keypharse to use
keyphrase = "keyphrase3"

model_name = dataset_name+"_"+keyphrase

time: 367 µs (started: 2024-08-16 14:12:42 +00:00)


## Creating the Graph

### Defining Graph Nodes and Edges 👀

- `Nodes` - documents and contexts
- `Edges`
  - document <- has -> context
- `Labels` - documents classes


#### Nodes and Edges 👀

##### Defining Docmuent nodes, Context nodes and edges between them

In [ ]:
all_contexts_list =[]
edges1,edges2 = [],[]
cont = 0
new_edge_cont = 0

document_nodes = []
context_nodes = []

edges_tuple = []

sentences = []
cont_sentences = 0
dit_sentences = {}

# Creating graph nodes and edges
for i in range(len(df)):
    # adding document nodes for every doc in df
    document_nodes.append(df["text_embeddings"][i])

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list: # if NOT
            # add a new context node for every new keyphrase in df
            context_nodes.append(df[keyphrase+"_embeddings"][i][j])
            # add a new edge between doc and new context
            edges1.append(df["id"][i])
            edges2.append(new_edge_cont)
            edges_tuple.append((df["id"][i],new_edge_cont))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges1.append(df["id"][i])
            edges2.append(all_contexts_list.index(key[0]))
            edges_tuple.append((df["id"][i],all_contexts_list.index(key[0])))
            cont+=1

    # organize sentences, sentences_embeddings, and a dict with the corresponding document for each sentence
    aux = df['sentences_embeddings'][i]
    for j,key in enumerate(aux):
        sentences.append(key)
        dit_sentences[cont_sentences] = df["id"][i]
        cont_sentences += 1


document_nodes = np.array(document_nodes)
context_nodes = np.array(context_nodes)

time: 14.4 s (started: 2024-08-16 14:12:43 +00:00)


In [ ]:
print("number of document nodes:",len(document_nodes))
print("number of context nodes:",len(context_nodes))
print("number of shared contexts:",cont)
print("number of direct edges (first dimension):",len(edges1))
print("number of direct edges (second dimension):",len(edges2))

number of document nodes: 10524
number of context nodes: 38370
number of shared contexts: 3961
number of direct edges (first dimension): 42331
number of direct edges (second dimension): 42331
time: 5.75 ms (started: 2024-08-16 14:12:57 +00:00)


In [ ]:
nbrs = NearestNeighbors(n_neighbors=15, metric='cosine').fit(context_nodes)

neighbors_list = nbrs.kneighbors(sentences, return_distance=False)

# cria aresta para cada vizinho encontrado
for i,neighbors in enumerate(neighbors_list):
        for n in neighbors:
            edges1.append(dit_sentences[i])
            edges2.append(n)
            edges_tuple.append((dit_sentences[i],n))

time: 10.2 s (started: 2024-08-16 14:12:57 +00:00)


##### Ajusting everything to Tensor Objects

In [ ]:
# transforms egdes to tensor
edges = np.array([edges1,edges2])
edges = torch.tensor(edges)

time: 22.1 ms (started: 2024-08-16 14:13:07 +00:00)


In [ ]:
# transforms nodes to tensor
document_nodes = np.array(document_nodes)
document_nodes = torch.tensor(document_nodes)
context_nodes = np.array(context_nodes)
context_nodes = torch.tensor(context_nodes)

time: 67.7 ms (started: 2024-08-16 14:13:07 +00:00)


In [ ]:
# Show documents nodes
document_nodes

tensor([[ 0.0480, -0.0557,  0.0054,  ..., -0.0087,  0.0842, -0.0266],
        [ 0.0155, -0.0795,  0.0167,  ..., -0.0797, -0.0875,  0.0055],
        [-0.0026, -0.0006, -0.0163,  ..., -0.0748,  0.0273, -0.0155],
        ...,
        [-0.0508,  0.0383, -0.0004,  ...,  0.0265, -0.0420, -0.0559],
        [-0.0322, -0.0649,  0.0155,  ...,  0.1090,  0.0247, -0.0226],
        [ 0.0748,  0.0328,  0.0163,  ..., -0.0506, -0.0447,  0.0613]])

time: 4.74 ms (started: 2024-08-16 14:13:07 +00:00)


In [ ]:
# Quantity of document nodes
len(document_nodes)

10524

time: 2.43 ms (started: 2024-08-16 14:13:07 +00:00)


In [ ]:
# Show context nodes
context_nodes

tensor([[ 0.0531, -0.0962,  0.0218,  ..., -0.0255,  0.0040, -0.0138],
        [ 0.0517, -0.0748,  0.0175,  ...,  0.0007,  0.0347, -0.0148],
        [ 0.0352,  0.0249,  0.0602,  ...,  0.1391,  0.0714, -0.0247],
        ...,
        [ 0.0899,  0.0222,  0.0036,  ..., -0.0822, -0.0003,  0.0316],
        [ 0.0639,  0.0573, -0.0116,  ...,  0.0225, -0.0384,  0.0333],
        [ 0.0135,  0.0573,  0.0189,  ..., -0.0285,  0.0146,  0.0765]])

time: 4.01 ms (started: 2024-08-16 14:13:07 +00:00)


In [ ]:
# Quantity of context nodes
len(context_nodes)

38370

time: 2.61 ms (started: 2024-08-16 14:13:07 +00:00)


In [ ]:
# Quantity of edges
print(len(edges[0]))
print(len(edges[1]))

201586
201586
time: 516 µs (started: 2024-08-16 14:13:07 +00:00)


In [ ]:
# showing edges
print(edges)

tensor([[    0,     0,     0,  ..., 10523, 10523, 10523],
        [    0,     1,     2,  ..., 38308, 37587, 37571]])
time: 1.18 ms (started: 2024-08-16 14:13:07 +00:00)


#### Class Labels

In [ ]:
# All different classes
print(df["class"].unique())

['data_management' 'ecology' 'economic' 'geophysics' 'gravitional_theory'
 'hydro' 'math' 'metals' 'networking' 'neuroscience' 'oceanography'
 'politic' 'sociology' 'software_engineering' 'statistics'
 'theory_computing']
time: 2.13 ms (started: 2024-08-16 14:13:07 +00:00)


In [ ]:
# Creating labels for classification
# the dictionary is a numeric representation for each possible "document" class

dit = {}
for i,classe in enumerate(df["class"].unique()):
  dit[classe] = i

print(dit,'\n')

labels = df[["class"]]
for i in range(len(df[["class"]])):
    labels["class"][i] = dit[labels["class"][i]]
labels.head()

{'data_management': 0, 'ecology': 1, 'economic': 2, 'geophysics': 3, 'gravitional_theory': 4, 'hydro': 5, 'math': 6, 'metals': 7, 'networking': 8, 'neuroscience': 9, 'oceanography': 10, 'politic': 11, 'sociology': 12, 'software_engineering': 13, 'statistics': 14, 'theory_computing': 15} 



<ipython-input-95-a018e46c97dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["class"][i] = dit[labels["class"][i]]


,class
0,0
1,0
2,0
3,0
4,0


time: 995 ms (started: 2024-08-16 14:13:07 +00:00)


In [ ]:
# tranfors class dataframe into tensor
y = labels["class"].tolist()
y = x_np = torch.tensor(y)
y

tensor([ 0,  0,  0,  ..., 15, 15, 15])

time: 6.46 ms (started: 2024-08-16 14:13:08 +00:00)


In [ ]:
len(y)

10524

time: 3.47 ms (started: 2024-08-16 14:13:08 +00:00)


### Testing Graph with Networkx




#### Defining overal graph in networkx maner

In [ ]:
# Run graph Representation for networkx
all_contexts_list =[]
edges_test = []
cont = 0
new_edge_cont = 0

document_nodes_test = []
context_nodes_test = []

# Creating graph nodes and edges
for i in range(len(df)):
    # adding new documents for every node
    document_nodes_test.append("doc_"+str(i)) # in the actual graph nodes -> documents embeddings

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list:
            # add a new context node for every new keyphrase in df
            context_nodes_test.append("contx_"+str(new_edge_cont)) # in the actual graph nodes -> context embeddings

            # add a new edge between doc and new context
            edges_test.append(("doc_"+str(df["id"][i]),"contx_"+str(new_edge_cont)))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges_test.append(("doc_"+str(df["id"][i]),"contx_"+str(all_contexts_list.index(key[0]))))
            cont+=1

time: 12.8 s (started: 2024-08-16 14:13:08 +00:00)


In [ ]:
edges_test = [("doc_"+str(i[0]),"contx_"+str(i[1])) for i in edges_tuple]

time: 133 ms (started: 2024-08-16 14:13:21 +00:00)


In [ ]:
print("number of document nodes:",len(document_nodes_test))
print("number of context nodes:",len(context_nodes_test))
print("number of edges:",len(edges_test))

number of document nodes: 10524
number of context nodes: 38370
number of edges: 201586
time: 4.56 ms (started: 2024-08-16 14:13:21 +00:00)


In [ ]:
print(edges_test[0:100])

[('doc_0', 'contx_0'), ('doc_0', 'contx_1'), ('doc_0', 'contx_2'), ('doc_0', 'contx_3'), ('doc_0', 'contx_4'), ('doc_1', 'contx_5'), ('doc_1', 'contx_6'), ('doc_1', 'contx_7'), ('doc_1', 'contx_8'), ('doc_2', 'contx_9'), ('doc_2', 'contx_10'), ('doc_2', 'contx_11'), ('doc_2', 'contx_12'), ('doc_3', 'contx_13'), ('doc_3', 'contx_14'), ('doc_3', 'contx_15'), ('doc_3', 'contx_16'), ('doc_3', 'contx_17'), ('doc_4', 'contx_18'), ('doc_4', 'contx_19'), ('doc_4', 'contx_20'), ('doc_4', 'contx_21'), ('doc_4', 'contx_22'), ('doc_5', 'contx_23'), ('doc_5', 'contx_24'), ('doc_6', 'contx_25'), ('doc_6', 'contx_26'), ('doc_6', 'contx_27'), ('doc_7', 'contx_28'), ('doc_7', 'contx_29'), ('doc_8', 'contx_30'), ('doc_8', 'contx_31'), ('doc_8', 'contx_32'), ('doc_8', 'contx_33'), ('doc_8', 'contx_34'), ('doc_9', 'contx_35'), ('doc_10', 'contx_36'), ('doc_10', 'contx_37'), ('doc_10', 'contx_38'), ('doc_10', 'contx_39'), ('doc_10', 'contx_40'), ('doc_11', 'contx_41'), ('doc_11', 'contx_42'), ('doc_11', 'c

#### Test graph Conectivity with networkx

In [ ]:
# Define Bipartide graph
B = nx.Graph()
B.add_nodes_from(document_nodes_test, bipartite=0)
B.add_nodes_from(context_nodes_test, bipartite=1)
B.add_edges_from(edges_test)

time: 394 ms (started: 2024-08-16 14:13:21 +00:00)


In [ ]:
nx.is_connected(B)

True

time: 113 ms (started: 2024-08-16 14:13:22 +00:00)


In [ ]:
# Number of conected elements
# if == 1 -> all elements of the graph are conected
print(nx.number_connected_components(B))

1
time: 125 ms (started: 2024-08-16 14:13:22 +00:00)


In [ ]:
# size of a cluster
print(len(nx.node_connected_component(B,'doc_0')))

48894
time: 92 ms (started: 2024-08-16 14:13:22 +00:00)


### Creating Graph HeteroData Object

In [ ]:
# Defining nodes, edges and class labels
from torch_geometric.data import HeteroData

data = HeteroData()

# nodes
data['document'].x = document_nodes
data['concept'].x = context_nodes

# edges
data['document', 'has', 'concept'].edge_index = edges

#class labels
data['document'].y = y


time: 664 µs (started: 2024-08-16 14:13:22 +00:00)


In [ ]:
# Setting graph to undirected
data = T.ToUndirected()(data)

time: 1.74 ms (started: 2024-08-16 14:13:22 +00:00)


In [ ]:
# removing duplicate edges
data = T.RemoveDuplicatedEdges()(data)

time: 32.9 ms (started: 2024-08-16 14:13:22 +00:00)


In [ ]:
# Ensure date in using gpu
data = data.to(device)

time: 19.2 ms (started: 2024-08-16 14:13:22 +00:00)


In [ ]:
data

HeteroData(
  document={
    x=[10524, 384],
    y=[10524],
  },
  concept={ x=[38370, 384] },
  (document, has, concept)={ edge_index=[2, 167583] },
  (concept, rev_has, document)={ edge_index=[2, 167583] }
)

time: 3.01 ms (started: 2024-08-16 14:13:22 +00:00)


## TRAINING

### Training rotulated base = 1 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',1)


=================== MODEL 0 ===================

Epoch: 001, Loss: 4.0329, Train: 0.9375, Test: 0.4499
Early stopping:  nan
Epoch: 002, Loss: 0.5202, Train: 1.0000, Test: 0.5810
Early stopping:  2.4838286329091774
Epoch: 003, Loss: 0.0411, Train: 1.0000, Test: 0.5817
Early stopping:  2.179561579391559
Epoch: 004, Loss: 0.0113, Train: 1.0000, Test: 0.5845
Early stopping:  1.9351033586258048
Epoch: 005, Loss: 0.0038, Train: 1.0000, Test: 0.5802
Early stopping:  1.7526724011720107
Epoch: 006, Loss: 0.0002, Train: 1.0000, Test: 0.5729
Early stopping:  0.226910760829561
Epoch: 007, Loss: 0.0000, Train: 1.0000, Test: 0.5679
Early stopping:  0.0172634282066465
Epoch: 008, Loss: 0.0000, Train: 1.0000, Test: 0.5652
Early stopping:  0.004886694805615043
PREDICTIONS -> tensor([ 0,  0,  0,  ..., 14,  2, 14], device='cuda:0')

Classification Report:

                      precision    recall  f1-score   support

     data_management       0.61      0.57      0.59       401
             ecology    

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 402 ms (started: 2024-08-16 14:13:25 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',1)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7849, Train: 1.0000, Test: 0.3588
Early stopping:  nan
Epoch: 002, Loss: 2.3904, Train: 1.0000, Test: 0.5153
Early stopping:  0.2789125019922387
Epoch: 003, Loss: 1.9320, Train: 1.0000, Test: 0.5448
Early stopping:  0.4268626248222807
Epoch: 004, Loss: 1.4255, Train: 1.0000, Test: 0.5674
Early stopping:  0.5865529875924703
Epoch: 005, Loss: 0.9414, Train: 1.0000, Test: 0.5826
Early stopping:  0.736293188638564
Epoch: 006, Loss: 0.5463, Train: 1.0000, Test: 0.5938
Early stopping:  0.7403746679234149
Epoch: 007, Loss: 0.2773, Train: 1.0000, Test: 0.5991
Early stopping:  0.666754974312117
Epoch: 008, Loss: 0.1275, Train: 1.0000, Test: 0.6038
Early stopping:  0.5263336341448823
Epoch: 009, Loss: 0.0571, Train: 1.0000, Test: 0.6045
Early stopping:  0.3608582587237114
Epoch: 010, Loss: 0.0266, Train: 1.0000, Test: 0.5993
Early stopping:  0.2129508427916691
Epoch: 011, Loss: 0.0134, Train: 1.0000, Test: 0.5954
Early stoppin

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 425 ms (started: 2024-08-16 14:13:33 +00:00)


### Training rotulated base = 5 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',5)


=================== MODEL 0 ===================

Epoch: 001, Loss: 3.5568, Train: 0.7750, Test: 0.6129
Early stopping:  nan
Epoch: 002, Loss: 0.8930, Train: 0.9500, Test: 0.7285
Early stopping:  1.8835798756190893
Epoch: 003, Loss: 0.2141, Train: 0.9750, Test: 0.7304
Early stopping:  1.766848725006492
Epoch: 004, Loss: 0.0837, Train: 1.0000, Test: 0.7153
Early stopping:  1.619288021087117
Epoch: 005, Loss: 0.0285, Train: 1.0000, Test: 0.7073
Early stopping:  1.4949737449455156
Epoch: 006, Loss: 0.0160, Train: 1.0000, Test: 0.7039
Early stopping:  0.3695205251866835
Epoch: 007, Loss: 0.0111, Train: 1.0000, Test: 0.7083
Early stopping:  0.08522124757679383
Epoch: 008, Loss: 0.0106, Train: 1.0000, Test: 0.7176
Early stopping:  0.0308848296023774
Epoch: 009, Loss: 0.0041, Train: 1.0000, Test: 0.7222
Early stopping:  0.009135104955824416
PREDICTIONS -> tensor([ 0,  0,  0,  ..., 15, 15, 15], device='cuda:0')

Classification Report:

                      precision    recall  f1-score   supp

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 413 ms (started: 2024-08-16 14:13:37 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',5)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7894, Train: 0.9125, Test: 0.6218
Early stopping:  nan
Epoch: 002, Loss: 2.4840, Train: 0.9250, Test: 0.7352
Early stopping:  0.2159518538127248
Epoch: 003, Loss: 2.1105, Train: 0.9375, Test: 0.7589
Early stopping:  0.3400251079935325
Epoch: 004, Loss: 1.6795, Train: 0.9375, Test: 0.7712
Early stopping:  0.4794505943041632
Epoch: 005, Loss: 1.2492, Train: 0.9625, Test: 0.7810
Early stopping:  0.6157347889560302
Epoch: 006, Loss: 0.8665, Train: 1.0000, Test: 0.7917
Early stopping:  0.6478815879585577
Epoch: 007, Loss: 0.5632, Train: 1.0000, Test: 0.7941
Early stopping:  0.6192819734760274
Epoch: 008, Loss: 0.3490, Train: 1.0000, Test: 0.7920
Early stopping:  0.5336295782149869
Epoch: 009, Loss: 0.2135, Train: 1.0000, Test: 0.7893
Early stopping:  0.41668834318836
Epoch: 010, Loss: 0.1314, Train: 1.0000, Test: 0.7894
Early stopping:  0.2961238160091948
Epoch: 011, Loss: 0.0812, Train: 1.0000, Test: 0.7893
Early stoppin

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 478 ms (started: 2024-08-16 14:13:46 +00:00)


### Training rotulated base = 10 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',10)


=================== MODEL 0 ===================

Epoch: 001, Loss: 3.2167, Train: 0.7625, Test: 0.7045
Early stopping:  nan
Epoch: 002, Loss: 0.7791, Train: 0.8562, Test: 0.7397
Early stopping:  1.7236664294396007
Epoch: 003, Loss: 0.4374, Train: 0.9437, Test: 0.7667
Early stopping:  1.515674895474295
Epoch: 004, Loss: 0.1864, Train: 0.9688, Test: 0.7791
Early stopping:  1.3958649880313438
Epoch: 005, Loss: 0.1320, Train: 0.9750, Test: 0.7788
Early stopping:  1.292507159065527
Epoch: 006, Loss: 0.1075, Train: 0.9875, Test: 0.7808
Early stopping:  0.28397333202377556
Epoch: 007, Loss: 0.0642, Train: 0.9875, Test: 0.7808
Early stopping:  0.14756869022929436
Epoch: 008, Loss: 0.0360, Train: 0.9938, Test: 0.7823
Early stopping:  0.05870465799957043
Epoch: 009, Loss: 0.0175, Train: 1.0000, Test: 0.7806
Early stopping:  0.04793504704416349
Epoch: 010, Loss: 0.0065, Train: 1.0000, Test: 0.7755
Early stopping:  0.04055840333760713
Epoch: 011, Loss: 0.0039, Train: 1.0000, Test: 0.7705
Early st

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 394 ms (started: 2024-08-16 14:13:52 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',10)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7811, Train: 0.7562, Test: 0.5229
Early stopping:  nan
Epoch: 002, Loss: 2.5169, Train: 0.9187, Test: 0.7213
Early stopping:  0.18677679374911968
Epoch: 003, Loss: 2.1877, Train: 0.9500, Test: 0.7701
Early stopping:  0.29728103424651514
Epoch: 004, Loss: 1.8045, Train: 0.9437, Test: 0.7782
Early stopping:  0.42211150633810185
Epoch: 005, Loss: 1.4139, Train: 0.9375, Test: 0.7776
Early stopping:  0.5465806113634537
Epoch: 006, Loss: 1.0544, Train: 0.9437, Test: 0.7761
Early stopping:  0.5850555770858353
Epoch: 007, Loss: 0.7564, Train: 0.9500, Test: 0.7725
Early stopping:  0.5719570732784492
Epoch: 008, Loss: 0.5324, Train: 0.9812, Test: 0.7720
Early stopping:  0.5090450094405661
Epoch: 009, Loss: 0.3737, Train: 0.9938, Test: 0.7757
Early stopping:  0.4163686046926992
Epoch: 010, Loss: 0.2629, Train: 1.0000, Test: 0.7770
Early stopping:  0.3163483541504974
Epoch: 011, Loss: 0.1850, Train: 1.0000, Test: 0.7801
Early st

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 408 ms (started: 2024-08-16 14:14:02 +00:00)


### Training rotulated base = 20 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',20)


=================== MODEL 0 ===================

Epoch: 001, Loss: 3.4796, Train: 0.7094, Test: 0.6378
Early stopping:  nan
Epoch: 002, Loss: 0.9156, Train: 0.8531, Test: 0.7708
Early stopping:  1.8130280321137329
Epoch: 003, Loss: 0.4565, Train: 0.9031, Test: 0.7839
Early stopping:  1.6291413628603246
Epoch: 004, Loss: 0.3436, Train: 0.9375, Test: 0.8040
Early stopping:  1.4747703580101723
Epoch: 005, Loss: 0.2236, Train: 0.9281, Test: 0.8047
Early stopping:  1.3647168531138527
Epoch: 006, Loss: 0.1638, Train: 0.9656, Test: 0.8004
Early stopping:  0.298754572435327
Epoch: 007, Loss: 0.1207, Train: 0.9812, Test: 0.8002
Early stopping:  0.13744078354530176
Epoch: 008, Loss: 0.0680, Train: 0.9875, Test: 0.7974
Early stopping:  0.10597077899919993
Epoch: 009, Loss: 0.0494, Train: 0.9875, Test: 0.7893
Early stopping:  0.071082891359334
Epoch: 010, Loss: 0.0410, Train: 0.9938, Test: 0.7864
Early stopping:  0.05223887179708905
Epoch: 011, Loss: 0.0231, Train: 0.9969, Test: 0.7849
Early stop

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 439 ms (started: 2024-08-16 14:14:08 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',20)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7839, Train: 0.8156, Test: 0.7076
Early stopping:  nan
Epoch: 002, Loss: 2.5159, Train: 0.8250, Test: 0.7453
Early stopping:  0.1895503935559246
Epoch: 003, Loss: 2.2081, Train: 0.8219, Test: 0.7497
Early stopping:  0.28813044581852804
Epoch: 004, Loss: 1.8535, Train: 0.8344, Test: 0.7592
Early stopping:  0.40085139890285804
Epoch: 005, Loss: 1.4898, Train: 0.8469, Test: 0.7707
Early stopping:  0.5149703756418026
Epoch: 006, Loss: 1.1562, Train: 0.8781, Test: 0.7809
Early stopping:  0.5437295800235419
Epoch: 007, Loss: 0.8791, Train: 0.9000, Test: 0.7895
Early stopping:  0.5311983254137532
Epoch: 008, Loss: 0.6651, Train: 0.9187, Test: 0.8021
Early stopping:  0.4748002338332647
Epoch: 009, Loss: 0.5069, Train: 0.9344, Test: 0.8048
Early stopping:  0.3924031164113588
Epoch: 010, Loss: 0.3949, Train: 0.9375, Test: 0.8052
Early stopping:  0.30402938491923204
Epoch: 011, Loss: 0.3181, Train: 0.9500, Test: 0.8047
Early st

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 431 ms (started: 2024-08-16 14:14:20 +00:00)


### Training rotulated base = 30 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',30)


=================== MODEL 0 ===================

Epoch: 001, Loss: 3.5498, Train: 0.7063, Test: 0.5860
Early stopping:  nan
Epoch: 002, Loss: 0.9020, Train: 0.8167, Test: 0.7355
Early stopping:  1.8722409825236581
Epoch: 003, Loss: 0.5234, Train: 0.8604, Test: 0.7768
Early stopping:  1.648885292043685
Epoch: 004, Loss: 0.4055, Train: 0.8896, Test: 0.7860
Early stopping:  1.4849079127435185
Epoch: 005, Loss: 0.3434, Train: 0.9104, Test: 0.8019
Early stopping:  1.361771650300464
Epoch: 006, Loss: 0.2522, Train: 0.9354, Test: 0.8071
Early stopping:  0.2529520034005417
Epoch: 007, Loss: 0.1894, Train: 0.9500, Test: 0.8073
Early stopping:  0.13066340652357367
Epoch: 008, Loss: 0.1643, Train: 0.9521, Test: 0.8098
Early stopping:  0.10215968140170152
Epoch: 009, Loss: 0.1410, Train: 0.9625, Test: 0.8188
Early stopping:  0.08142715240295695
Epoch: 010, Loss: 0.1096, Train: 0.9729, Test: 0.8186
Early stopping:  0.05395142302027506
Epoch: 011, Loss: 0.0849, Train: 0.9833, Test: 0.8134
Early sto

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 477 ms (started: 2024-08-16 14:14:28 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',30)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7895, Train: 0.6875, Test: 0.6507
Early stopping:  nan
Epoch: 002, Loss: 2.5234, Train: 0.7979, Test: 0.7442
Early stopping:  0.18815735591894253
Epoch: 003, Loss: 2.2002, Train: 0.8375, Test: 0.7665
Early stopping:  0.29513687383457293
Epoch: 004, Loss: 1.8254, Train: 0.8542, Test: 0.7719
Early stopping:  0.4163118496928804
Epoch: 005, Loss: 1.4530, Train: 0.8604, Test: 0.7814
Early stopping:  0.5342854083405189
Epoch: 006, Loss: 1.1192, Train: 0.8667, Test: 0.7969
Early stopping:  0.5624106934632878
Epoch: 007, Loss: 0.8435, Train: 0.8667, Test: 0.8071
Early stopping:  0.5417021186916977
Epoch: 008, Loss: 0.6361, Train: 0.8729, Test: 0.8127
Early stopping:  0.4753286507184025
Epoch: 009, Loss: 0.4928, Train: 0.8771, Test: 0.8187
Early stopping:  0.384713244783109
Epoch: 010, Loss: 0.3956, Train: 0.8917, Test: 0.8195
Early stopping:  0.2897947700801969
Epoch: 011, Loss: 0.3284, Train: 0.9083, Test: 0.8231
Early stop

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 438 ms (started: 2024-08-16 14:14:40 +00:00)


### Training rotulated base = 80% ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN','80%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 5.4239, Train: 0.1653, Test: 0.1721
Early stopping:  nan
Epoch: 002, Loss: 12.5963, Train: 0.1701, Test: 0.1716
Early stopping:  5.0716402002967955
Epoch: 003, Loss: 12.6517, Train: 0.2808, Test: 0.2779
Early stopping:  4.157081515254105
Epoch: 004, Loss: 9.7003, Train: 0.3666, Test: 0.3700
Early stopping:  3.4043249504935438
Epoch: 005, Loss: 6.1940, Train: 0.3171, Test: 0.3063
Early stopping:  3.4252941513867863
Epoch: 006, Loss: 6.8460, Train: 0.3470, Test: 0.3395
Early stopping:  3.061305589056082
Epoch: 007, Loss: 5.9098, Train: 0.3689, Test: 0.3489
Early stopping:  2.8789463899110386
Epoch: 008, Loss: 4.6041, Train: 0.4345, Test: 0.4068
Early stopping:  1.8897503744647153
Epoch: 009, Loss: 3.3294, Train: 0.4780, Test: 0.4658
Early stopping:  1.4052968555629086
Epoch: 010, Loss: 2.5441, Train: 0.4861, Test: 0.4742
Early stopping:  1.7741348707177251
Epoch: 011, Loss: 2.1945, Train: 0.4786, Test: 0.4742
Early stopp

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 396 ms (started: 2024-08-17 14:17:42 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT','80%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.9516, Train: 0.3708, Test: 0.3642
Early stopping:  nan
Epoch: 002, Loss: 2.8218, Train: 0.5242, Test: 0.5100
Early stopping:  0.09182567602547291
Epoch: 003, Loss: 2.6574, Train: 0.5462, Test: 0.5168
Early stopping:  0.1474472939959434
Epoch: 004, Loss: 2.4685, Train: 0.5671, Test: 0.5389
Early stopping:  0.20904086312435768
Epoch: 005, Loss: 2.2699, Train: 0.5899, Test: 0.5705
Early stopping:  0.27232267987651354
Epoch: 006, Loss: 2.0711, Train: 0.6104, Test: 0.5905
Early stopping:  0.29885855196152794
Epoch: 007, Loss: 1.8796, Train: 0.6309, Test: 0.6184
Early stopping:  0.3088029222987719
Epoch: 008, Loss: 1.7019, Train: 0.6404, Test: 0.6263
Early stopping:  0.30419721714024717
Epoch: 009, Loss: 1.5475, Train: 0.6449, Test: 0.6311
Early stopping:  0.2871609024585015
Epoch: 010, Loss: 1.4215, Train: 0.6512, Test: 0.6321
Early stopping:  0.25878232986934796
Epoch: 011, Loss: 1.3216, Train: 0.6562, Test: 0.6411
Early

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 515 ms (started: 2024-08-17 14:18:22 +00:00)


--------------------------------------
# Hetrogenous Graph for Keyphrase = 2 or 3
--------------------------------------

In [ ]:
print(df["class"].unique())
class_number = len(df["class"].unique())
print(class_number)

['data_management' 'ecology' 'economic' 'geophysics' 'gravitional_theory'
 'hydro' 'math' 'metals' 'networking' 'neuroscience' 'oceanography'
 'politic' 'sociology' 'software_engineering' 'statistics'
 'theory_computing']
16
time: 9 ms (started: 2024-08-16 14:14:41 +00:00)


In [ ]:
# Change here to change wich keypharse to use
keyphrase = "keyphrase23"

model_name = dataset_name+"_"+keyphrase

time: 453 µs (started: 2024-08-16 14:14:41 +00:00)


## Creating the Graph

### Defining Graph Nodes and Edges 👀

- `Nodes` - documents and contexts
- `Edges`
  - document <- has -> context
- `Labels` - documents classes


#### Nodes and Edges 👀

##### Defining Docmuent nodes, Context nodes and edges between them

In [ ]:
all_contexts_list =[]
edges1,edges2 = [],[]
cont = 0
new_edge_cont = 0

document_nodes = []
context_nodes = []

edges_tuple = []

sentences = []
cont_sentences = 0
dit_sentences = {}

# Creating graph nodes and edges
for i in range(len(df)):
    # adding document nodes for every doc in df
    document_nodes.append(df["text_embeddings"][i])

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list: # if NOT
            # add a new context node for every new keyphrase in df
            context_nodes.append(df[keyphrase+"_embeddings"][i][j])
            # add a new edge between doc and new context
            edges1.append(df["id"][i])
            edges2.append(new_edge_cont)
            edges_tuple.append((df["id"][i],new_edge_cont))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges1.append(df["id"][i])
            edges2.append(all_contexts_list.index(key[0]))
            edges_tuple.append((df["id"][i],all_contexts_list.index(key[0])))
            cont+=1

    # organize sentences, sentences_embeddings, and a dict with the corresponding document for each sentence
    aux = df['sentences_embeddings'][i]
    for j,key in enumerate(aux):
        sentences.append(key)
        dit_sentences[cont_sentences] = df["id"][i]
        cont_sentences += 1


document_nodes = np.array(document_nodes)
context_nodes = np.array(context_nodes)

time: 21.6 s (started: 2024-08-16 14:14:41 +00:00)


In [ ]:
print("number of document nodes:",len(document_nodes))
print("number of context nodes:",len(context_nodes))
print("number of shared contexts:",cont)
print("number of direct edges (first dimension):",len(edges1))
print("number of direct edges (second dimension):",len(edges2))

number of document nodes: 10524
number of context nodes: 44786
number of shared contexts: 6772
number of direct edges (first dimension): 51558
number of direct edges (second dimension): 51558
time: 4.75 ms (started: 2024-08-16 14:15:02 +00:00)


In [ ]:
nbrs = NearestNeighbors(n_neighbors=15, metric='cosine').fit(context_nodes)

neighbors_list = nbrs.kneighbors(sentences, return_distance=False)

# cria aresta para cada vizinho encontrado
for i,neighbors in enumerate(neighbors_list):
        for n in neighbors:
            edges1.append(dit_sentences[i])
            edges2.append(n)
            edges_tuple.append((dit_sentences[i],n))

time: 10.5 s (started: 2024-08-16 14:15:02 +00:00)


##### Ajusting everything to Tensor Objects

In [ ]:
# transforms egdes to tensor
edges = np.array([edges1,edges2])
edges = torch.tensor(edges)

time: 24.8 ms (started: 2024-08-16 14:15:13 +00:00)


In [ ]:
# transforms nodes to tensor
document_nodes = np.array(document_nodes)
document_nodes = torch.tensor(document_nodes)
context_nodes = np.array(context_nodes)
context_nodes = torch.tensor(context_nodes)

time: 77.1 ms (started: 2024-08-16 14:15:13 +00:00)


In [ ]:
# Show documents nodes
document_nodes

tensor([[ 0.0480, -0.0557,  0.0054,  ..., -0.0087,  0.0842, -0.0266],
        [ 0.0155, -0.0795,  0.0167,  ..., -0.0797, -0.0875,  0.0055],
        [-0.0026, -0.0006, -0.0163,  ..., -0.0748,  0.0273, -0.0155],
        ...,
        [-0.0508,  0.0383, -0.0004,  ...,  0.0265, -0.0420, -0.0559],
        [-0.0322, -0.0649,  0.0155,  ...,  0.1090,  0.0247, -0.0226],
        [ 0.0748,  0.0328,  0.0163,  ..., -0.0506, -0.0447,  0.0613]])

time: 4.23 ms (started: 2024-08-16 14:15:13 +00:00)


In [ ]:
# Quantity of document nodes
len(document_nodes)

10524

time: 2.4 ms (started: 2024-08-16 14:15:13 +00:00)


In [ ]:
# Show context nodes
context_nodes

tensor([[ 0.0531, -0.0962,  0.0218,  ..., -0.0255,  0.0040, -0.0138],
        [ 0.0517, -0.0748,  0.0175,  ...,  0.0007,  0.0347, -0.0148],
        [ 0.0352,  0.0249,  0.0602,  ...,  0.1391,  0.0714, -0.0247],
        ...,
        [ 0.0433,  0.0305, -0.0150,  ..., -0.0104, -0.0447,  0.0464],
        [ 0.0851,  0.0426, -0.0437,  ..., -0.0166, -0.0241, -0.0064],
        [ 0.0135,  0.0573,  0.0189,  ..., -0.0285,  0.0146,  0.0765]])

time: 4.64 ms (started: 2024-08-16 14:15:13 +00:00)


In [ ]:
# Quantity of context nodes
len(context_nodes)

44786

time: 2.95 ms (started: 2024-08-16 14:15:13 +00:00)


In [ ]:
# Quantity of edges
print(len(edges[0]))
print(len(edges[1]))

210813
210813
time: 564 µs (started: 2024-08-16 14:15:13 +00:00)


In [ ]:
# showing edges
print(edges)

tensor([[    0,     0,     0,  ..., 10523, 10523, 10523],
        [    0,     1,     2,  ..., 43530, 43845, 44327]])
time: 2.21 ms (started: 2024-08-16 14:15:13 +00:00)


#### Class Labels

In [ ]:
# All different classes
print(df["class"].unique())

['data_management' 'ecology' 'economic' 'geophysics' 'gravitional_theory'
 'hydro' 'math' 'metals' 'networking' 'neuroscience' 'oceanography'
 'politic' 'sociology' 'software_engineering' 'statistics'
 'theory_computing']
time: 1.37 ms (started: 2024-08-16 14:15:13 +00:00)


In [ ]:
# Creating labels for classification
# the dictionary is a numeric representation for each possible "document" class

dit = {}
for i,classe in enumerate(df["class"].unique()):
  dit[classe] = i

print(dit,'\n')

labels = df[["class"]]
for i in range(len(df[["class"]])):
    labels["class"][i] = dit[labels["class"][i]]
labels.head()

{'data_management': 0, 'ecology': 1, 'economic': 2, 'geophysics': 3, 'gravitional_theory': 4, 'hydro': 5, 'math': 6, 'metals': 7, 'networking': 8, 'neuroscience': 9, 'oceanography': 10, 'politic': 11, 'sociology': 12, 'software_engineering': 13, 'statistics': 14, 'theory_computing': 15} 



<ipython-input-145-a018e46c97dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["class"][i] = dit[labels["class"][i]]


,class
0,0
1,0
2,0
3,0
4,0


time: 974 ms (started: 2024-08-16 14:15:13 +00:00)


In [ ]:
# tranfors class dataframe into tensor
y = labels["class"].tolist()
y = x_np = torch.tensor(y)
y

tensor([ 0,  0,  0,  ..., 15, 15, 15])

time: 7.19 ms (started: 2024-08-16 14:15:14 +00:00)


In [ ]:
len(y)

10524

time: 2.85 ms (started: 2024-08-16 14:15:14 +00:00)


### Testing Graph with Networkx




#### Defining overal graph in networkx maner

In [ ]:
# Run graph Representation for networkx
all_contexts_list =[]
edges_test = []
cont = 0
new_edge_cont = 0

document_nodes_test = []
context_nodes_test = []

# Creating graph nodes and edges
for i in range(len(df)):
    # adding new documents for every node
    document_nodes_test.append("doc_"+str(i)) # in the actual graph nodes -> documents embeddings

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list:
            # add a new context node for every new keyphrase in df
            context_nodes_test.append("contx_"+str(new_edge_cont)) # in the actual graph nodes -> context embeddings

            # add a new edge between doc and new context
            edges_test.append(("doc_"+str(df["id"][i]),"contx_"+str(new_edge_cont)))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges_test.append(("doc_"+str(df["id"][i]),"contx_"+str(all_contexts_list.index(key[0]))))
            cont+=1

time: 18 s (started: 2024-08-16 14:15:14 +00:00)


In [ ]:
edges_test = [("doc_"+str(i[0]),"contx_"+str(i[1])) for i in edges_tuple]

time: 133 ms (started: 2024-08-16 14:15:32 +00:00)


In [ ]:
print("number of document nodes:",len(document_nodes_test))
print("number of context nodes:",len(context_nodes_test))
print("number of edges:",len(edges_test))

number of document nodes: 10524
number of context nodes: 44786
number of edges: 210813
time: 1.1 ms (started: 2024-08-16 14:15:32 +00:00)


In [ ]:
print(edges_test[0:100])

[('doc_0', 'contx_0'), ('doc_0', 'contx_1'), ('doc_0', 'contx_2'), ('doc_0', 'contx_3'), ('doc_0', 'contx_4'), ('doc_1', 'contx_5'), ('doc_1', 'contx_6'), ('doc_1', 'contx_7'), ('doc_1', 'contx_8'), ('doc_1', 'contx_9'), ('doc_2', 'contx_10'), ('doc_2', 'contx_11'), ('doc_2', 'contx_12'), ('doc_2', 'contx_13'), ('doc_2', 'contx_14'), ('doc_3', 'contx_15'), ('doc_3', 'contx_16'), ('doc_3', 'contx_17'), ('doc_3', 'contx_18'), ('doc_3', 'contx_19'), ('doc_4', 'contx_20'), ('doc_4', 'contx_21'), ('doc_4', 'contx_22'), ('doc_4', 'contx_23'), ('doc_4', 'contx_24'), ('doc_5', 'contx_25'), ('doc_5', 'contx_26'), ('doc_5', 'contx_27'), ('doc_5', 'contx_28'), ('doc_5', 'contx_29'), ('doc_6', 'contx_30'), ('doc_6', 'contx_31'), ('doc_6', 'contx_32'), ('doc_6', 'contx_33'), ('doc_6', 'contx_34'), ('doc_7', 'contx_35'), ('doc_7', 'contx_36'), ('doc_7', 'contx_37'), ('doc_7', 'contx_38'), ('doc_7', 'contx_39'), ('doc_8', 'contx_40'), ('doc_8', 'contx_41'), ('doc_8', 'contx_42'), ('doc_8', 'contx_43'

#### Test graph Conectivity with networkx

In [ ]:
# Define Bipartide graph
B = nx.Graph()
B.add_nodes_from(document_nodes_test, bipartite=0)
B.add_nodes_from(context_nodes_test, bipartite=1)
B.add_edges_from(edges_test)

time: 400 ms (started: 2024-08-16 14:15:32 +00:00)


In [ ]:
nx.is_connected(B)

True

time: 111 ms (started: 2024-08-16 14:15:33 +00:00)


In [ ]:
# Number of conected elements
# if == 1 -> all elements of the graph are conected
print(nx.number_connected_components(B))

1
time: 144 ms (started: 2024-08-16 14:15:33 +00:00)


In [ ]:
# size of a cluster
print(len(nx.node_connected_component(B,'doc_0')))

55310
time: 92.3 ms (started: 2024-08-16 14:15:33 +00:00)


### Creating Graph HeteroData Object

In [ ]:
# Defining nodes, edges and class labels
from torch_geometric.data import HeteroData

data = HeteroData()

# nodes
data['document'].x = document_nodes
data['concept'].x = context_nodes

# edges
data['document', 'has', 'concept'].edge_index = edges

#class labels
data['document'].y = y


time: 1.12 ms (started: 2024-08-16 14:15:33 +00:00)


In [ ]:
# Setting graph to undirected
data = T.ToUndirected()(data)

time: 1.58 ms (started: 2024-08-16 14:15:33 +00:00)


In [ ]:
# removing duplicate edges
data = T.RemoveDuplicatedEdges()(data)

time: 41.2 ms (started: 2024-08-16 14:15:33 +00:00)


In [ ]:
# Ensure date in using gpu
data = data.to(device)

time: 20.8 ms (started: 2024-08-16 14:15:33 +00:00)


In [ ]:
data

HeteroData(
  document={
    x=[10524, 384],
    y=[10524],
  },
  concept={ x=[44786, 384] },
  (document, has, concept)={ edge_index=[2, 165253] },
  (concept, rev_has, document)={ edge_index=[2, 165253] }
)

time: 2.99 ms (started: 2024-08-16 14:15:33 +00:00)


## TRAINING

### Training rotulated base = 1 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',1)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7656, Train: 0.9375, Test: 0.4432
Early stopping:  nan
Epoch: 002, Loss: 0.3733, Train: 0.9375, Test: 0.5634
Early stopping:  1.6916215483140404
Epoch: 003, Loss: 0.1518, Train: 1.0000, Test: 0.6053
Early stopping:  1.4493726572604395
Epoch: 004, Loss: 0.0026, Train: 0.9375, Test: 0.5908
Early stopping:  1.3037733709508545
Epoch: 005, Loss: 0.1747, Train: 1.0000, Test: 0.5805
Early stopping:  1.1657732102720948
Epoch: 006, Loss: 0.0017, Train: 1.0000, Test: 0.5763
Early stopping:  0.1531017888688253
Epoch: 007, Loss: 0.0001, Train: 1.0000, Test: 0.5694
Early stopping:  0.08897229387521467
Epoch: 008, Loss: 0.0002, Train: 1.0000, Test: 0.5629
Early stopping:  0.07760282685470796
Epoch: 009, Loss: 0.0002, Train: 1.0000, Test: 0.5602
Early stopping:  0.07787025130847951
Epoch: 010, Loss: 0.0002, Train: 1.0000, Test: 0.5586
Early stopping:  0.0006975922754950241
PREDICTIONS -> tensor([0, 0, 0,  ..., 0, 0, 0], device='cud

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 442 ms (started: 2024-08-16 14:15:37 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',1)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7618, Train: 1.0000, Test: 0.4900
Early stopping:  nan
Epoch: 002, Loss: 2.3102, Train: 1.0000, Test: 0.5280
Early stopping:  0.31939168400125373
Epoch: 003, Loss: 1.8120, Train: 1.0000, Test: 0.5318
Early stopping:  0.47509662924031393
Epoch: 004, Loss: 1.2791, Train: 1.0000, Test: 0.5379
Early stopping:  0.6390080506841475
Epoch: 005, Loss: 0.7960, Train: 1.0000, Test: 0.5491
Early stopping:  0.7849577588815857
Epoch: 006, Loss: 0.4296, Train: 1.0000, Test: 0.5589
Early stopping:  0.7568808326612992
Epoch: 007, Loss: 0.2032, Train: 1.0000, Test: 0.5695
Early stopping:  0.6506006240324268
Epoch: 008, Loss: 0.0897, Train: 1.0000, Test: 0.5736
Early stopping:  0.48432877697212345
Epoch: 009, Loss: 0.0403, Train: 1.0000, Test: 0.5799
Early stopping:  0.3094927819544455
Epoch: 010, Loss: 0.0192, Train: 1.0000, Test: 0.5817
Early stopping:  0.16850720823904503
Epoch: 011, Loss: 0.0097, Train: 1.0000, Test: 0.5838
Early s

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 416 ms (started: 2024-08-16 14:15:44 +00:00)


### Training rotulated base = 5 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',5)


=================== MODEL 0 ===================

Epoch: 001, Loss: 3.5453, Train: 0.9250, Test: 0.7193
Early stopping:  nan
Epoch: 002, Loss: 0.5474, Train: 0.9750, Test: 0.7545
Early stopping:  2.1198567099514047
Epoch: 003, Loss: 0.1191, Train: 1.0000, Test: 0.7479
Early stopping:  1.8668255554496043
Epoch: 004, Loss: 0.0285, Train: 1.0000, Test: 0.7349
Early stopping:  1.6722035551627097
Epoch: 005, Loss: 0.0103, Train: 1.0000, Test: 0.7225
Early stopping:  1.522364452423201
Epoch: 006, Loss: 0.0047, Train: 1.0000, Test: 0.7136
Early stopping:  0.2312532348403261
Epoch: 007, Loss: 0.0021, Train: 1.0000, Test: 0.7096
Early stopping:  0.049224984983880185
Epoch: 008, Loss: 0.0009, Train: 1.0000, Test: 0.7046
Early stopping:  0.01133496485395092
Epoch: 009, Loss: 0.0005, Train: 1.0000, Test: 0.7022
Early stopping:  0.004041564476810928
PREDICTIONS -> tensor([ 0,  0,  0,  ..., 15, 15, 15], device='cuda:0')

Classification Report:

                      precision    recall  f1-score   s

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 407 ms (started: 2024-08-16 14:15:48 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',5)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7588, Train: 0.9000, Test: 0.6344
Early stopping:  nan
Epoch: 002, Loss: 2.4537, Train: 0.9625, Test: 0.7415
Early stopping:  0.2157154942862757
Epoch: 003, Loss: 2.0785, Train: 0.9875, Test: 0.7600
Early stopping:  0.34076605373258784
Epoch: 004, Loss: 1.6498, Train: 0.9875, Test: 0.7639
Early stopping:  0.4792950372445119
Epoch: 005, Loss: 1.2206, Train: 0.9875, Test: 0.7695
Early stopping:  0.6149638617779677
Epoch: 006, Loss: 0.8394, Train: 1.0000, Test: 0.7781
Early stopping:  0.6463595469435263
Epoch: 007, Loss: 0.5370, Train: 1.0000, Test: 0.7875
Early stopping:  0.6170511444039583
Epoch: 008, Loss: 0.3219, Train: 1.0000, Test: 0.7938
Early stopping:  0.5323859250279785
Epoch: 009, Loss: 0.1854, Train: 1.0000, Test: 0.7939
Early stopping:  0.416388166294593
Epoch: 010, Loss: 0.1063, Train: 1.0000, Test: 0.7919
Early stopping:  0.29588486548539866
Epoch: 011, Loss: 0.0624, Train: 1.0000, Test: 0.7900
Early stop

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 429 ms (started: 2024-08-16 14:15:58 +00:00)


### Training rotulated base = 10 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',10)


=================== MODEL 0 ===================

Epoch: 001, Loss: 3.3292, Train: 0.7000, Test: 0.6100
Early stopping:  nan
Epoch: 002, Loss: 0.9907, Train: 0.8750, Test: 0.7355
Early stopping:  1.6535387939645063
Epoch: 003, Loss: 0.3939, Train: 0.9500, Test: 0.7701
Early stopping:  1.5513601312953795
Epoch: 004, Loss: 0.1679, Train: 0.9812, Test: 0.7881
Early stopping:  1.4480469592266985
Epoch: 005, Loss: 0.0746, Train: 1.0000, Test: 0.7830
Early stopping:  1.3546997811835928
Epoch: 006, Loss: 0.0355, Train: 1.0000, Test: 0.7771
Early stopping:  0.3933338149205471
Epoch: 007, Loss: 0.0189, Train: 1.0000, Test: 0.7750
Early stopping:  0.1541980734510987
Epoch: 008, Loss: 0.0111, Train: 1.0000, Test: 0.7707
Early stopping:  0.064274221770051
Epoch: 009, Loss: 0.0066, Train: 1.0000, Test: 0.7669
Early stopping:  0.02759376930905674
Epoch: 010, Loss: 0.0042, Train: 1.0000, Test: 0.7622
Early stopping:  0.012602688417282295
Epoch: 011, Loss: 0.0026, Train: 1.0000, Test: 0.7587
Early sto

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 427 ms (started: 2024-08-16 14:16:03 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',10)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7781, Train: 0.7438, Test: 0.5773
Early stopping:  nan
Epoch: 002, Loss: 2.5157, Train: 0.9000, Test: 0.6702
Early stopping:  0.1855720682312843
Epoch: 003, Loss: 2.1878, Train: 0.9062, Test: 0.7025
Early stopping:  0.2957661203934359
Epoch: 004, Loss: 1.8059, Train: 0.9125, Test: 0.7299
Early stopping:  0.4202932513092133
Epoch: 005, Loss: 1.4198, Train: 0.9437, Test: 0.7603
Early stopping:  0.5433501357021463
Epoch: 006, Loss: 1.0674, Train: 0.9625, Test: 0.7851
Early stopping:  0.579634714024966
Epoch: 007, Loss: 0.7706, Train: 0.9688, Test: 0.7925
Early stopping:  0.5656568226307641
Epoch: 008, Loss: 0.5426, Train: 0.9812, Test: 0.7904
Early stopping:  0.5046114622497601
Epoch: 009, Loss: 0.3825, Train: 0.9875, Test: 0.7894
Early stopping:  0.41544257416037994
Epoch: 010, Loss: 0.2720, Train: 0.9938, Test: 0.7889
Early stopping:  0.31838275466707744
Epoch: 011, Loss: 0.1924, Train: 1.0000, Test: 0.7895
Early stop

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 430 ms (started: 2024-08-16 14:16:13 +00:00)


### Training rotulated base = 20 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',20)


=================== MODEL 0 ===================

Epoch: 001, Loss: 3.7181, Train: 0.7156, Test: 0.5747
Early stopping:  nan
Epoch: 002, Loss: 1.1178, Train: 0.8469, Test: 0.7544
Early stopping:  1.838644043701684
Epoch: 003, Loss: 0.5799, Train: 0.8875, Test: 0.7926
Early stopping:  1.6782114427483215
Epoch: 004, Loss: 0.3312, Train: 0.9219, Test: 0.7905
Early stopping:  1.5559067449060915
Epoch: 005, Loss: 0.2099, Train: 0.9375, Test: 0.7839
Early stopping:  1.4548770497208448
Epoch: 006, Loss: 0.1939, Train: 0.9531, Test: 0.7855
Early stopping:  0.3852080489587829
Epoch: 007, Loss: 0.1375, Train: 0.9719, Test: 0.7902
Early stopping:  0.176523814848216
Epoch: 008, Loss: 0.0763, Train: 0.9812, Test: 0.7918
Early stopping:  0.09488459365434715
Epoch: 009, Loss: 0.0520, Train: 0.9875, Test: 0.7890
Early stopping:  0.06966326106104957
Epoch: 010, Loss: 0.0423, Train: 0.9969, Test: 0.7865
Early stopping:  0.06406798051526205
Epoch: 011, Loss: 0.0321, Train: 0.9969, Test: 0.7817
Early stop

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 474 ms (started: 2024-08-16 14:16:19 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',20)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7791, Train: 0.7375, Test: 0.6586
Early stopping:  nan
Epoch: 002, Loss: 2.5292, Train: 0.8156, Test: 0.7262
Early stopping:  0.17669375029876744
Epoch: 003, Loss: 2.2172, Train: 0.8500, Test: 0.7397
Early stopping:  0.28149652359518845
Epoch: 004, Loss: 1.8498, Train: 0.8656, Test: 0.7494
Early stopping:  0.40160073777164906
Epoch: 005, Loss: 1.4795, Train: 0.8781, Test: 0.7689
Early stopping:  0.5199853074582157
Epoch: 006, Loss: 1.1438, Train: 0.9031, Test: 0.7868
Early stopping:  0.5549644831640589
Epoch: 007, Loss: 0.8626, Train: 0.9062, Test: 0.8015
Early stopping:  0.5407905619458054
Epoch: 008, Loss: 0.6445, Train: 0.9250, Test: 0.8052
Early stopping:  0.48113196278332426
Epoch: 009, Loss: 0.4890, Train: 0.9313, Test: 0.8043
Early stopping:  0.3962474832537484
Epoch: 010, Loss: 0.3826, Train: 0.9437, Test: 0.8043
Early stopping:  0.30481530234098675
Epoch: 011, Loss: 0.3063, Train: 0.9531, Test: 0.8076
Early 

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 408 ms (started: 2024-08-16 14:16:31 +00:00)


### Training rotulated base = 30 ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',30)


=================== MODEL 0 ===================

Epoch: 001, Loss: 3.1956, Train: 0.7292, Test: 0.6424
Early stopping:  nan
Epoch: 002, Loss: 1.0072, Train: 0.8792, Test: 0.7892
Early stopping:  1.5474217959585654
Epoch: 003, Loss: 0.4179, Train: 0.8958, Test: 0.7959
Early stopping:  1.4635604912794384
Epoch: 004, Loss: 0.3117, Train: 0.9042, Test: 0.8056
Early stopping:  1.3436327421728869
Epoch: 005, Loss: 0.2364, Train: 0.9313, Test: 0.8048
Early stopping:  1.2460640983461424
Epoch: 006, Loss: 0.1841, Train: 0.9583, Test: 0.8109
Early stopping:  0.33362976503157254
Epoch: 007, Loss: 0.1232, Train: 0.9729, Test: 0.8124
Early stopping:  0.11453256495284614
Epoch: 008, Loss: 0.0795, Train: 0.9896, Test: 0.8074
Early stopping:  0.09166117000713782
Epoch: 009, Loss: 0.0556, Train: 0.9896, Test: 0.8035
Early stopping:  0.07452725803192566
Epoch: 010, Loss: 0.0441, Train: 0.9958, Test: 0.7993
Early stopping:  0.057214121596334225
Epoch: 011, Loss: 0.0310, Train: 1.0000, Test: 0.7980
Early

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 436 ms (started: 2024-08-16 14:16:38 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',30)


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.7767, Train: 0.7458, Test: 0.6235
Early stopping:  nan
Epoch: 002, Loss: 2.4904, Train: 0.8500, Test: 0.7553
Early stopping:  0.20247346024633492
Epoch: 003, Loss: 2.1474, Train: 0.8583, Test: 0.7702
Early stopping:  0.31509806393836925
Epoch: 004, Loss: 1.7572, Train: 0.8708, Test: 0.7766
Early stopping:  0.4401836050940379
Epoch: 005, Loss: 1.3748, Train: 0.8812, Test: 0.7888
Early stopping:  0.5602824452128944
Epoch: 006, Loss: 1.0398, Train: 0.8812, Test: 0.7975
Early stopping:  0.5810817295928685
Epoch: 007, Loss: 0.7726, Train: 0.8875, Test: 0.8056
Early stopping:  0.5496551012478964
Epoch: 008, Loss: 0.5768, Train: 0.8979, Test: 0.8091
Early stopping:  0.4721709892906704
Epoch: 009, Loss: 0.4442, Train: 0.9083, Test: 0.8114
Early stopping:  0.3729464937993651
Epoch: 010, Loss: 0.3572, Train: 0.9083, Test: 0.8127
Early stopping:  0.27371908855402627
Epoch: 011, Loss: 0.2971, Train: 0.9208, Test: 0.8137
Early st

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 471 ms (started: 2024-08-16 14:16:51 +00:00)


### Training rotulated base = 80% ❎

#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN','80%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 5.4239, Train: 0.1653, Test: 0.1721
Early stopping:  nan
Epoch: 002, Loss: 12.5963, Train: 0.1701, Test: 0.1716
Early stopping:  5.0716402002967955
Epoch: 003, Loss: 12.6517, Train: 0.2808, Test: 0.2779
Early stopping:  4.157081515254105
Epoch: 004, Loss: 9.7003, Train: 0.3666, Test: 0.3700
Early stopping:  3.4043249504935438
Epoch: 005, Loss: 6.1940, Train: 0.3171, Test: 0.3063
Early stopping:  3.4252941513867863
Epoch: 006, Loss: 6.8460, Train: 0.3470, Test: 0.3395
Early stopping:  3.061305589056082
Epoch: 007, Loss: 5.9098, Train: 0.3689, Test: 0.3489
Early stopping:  2.8789463899110386
Epoch: 008, Loss: 4.6041, Train: 0.4345, Test: 0.4068
Early stopping:  1.8897503744647153
Epoch: 009, Loss: 3.3294, Train: 0.4780, Test: 0.4658
Early stopping:  1.4052968555629086
Epoch: 010, Loss: 2.5441, Train: 0.4861, Test: 0.4742
Early stopping:  1.7741348707177251
Epoch: 011, Loss: 2.1945, Train: 0.4786, Test: 0.4742
Early stopp

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 396 ms (started: 2024-08-17 14:17:42 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT','80%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.9516, Train: 0.3708, Test: 0.3642
Early stopping:  nan
Epoch: 002, Loss: 2.8218, Train: 0.5242, Test: 0.5100
Early stopping:  0.09182567602547291
Epoch: 003, Loss: 2.6574, Train: 0.5462, Test: 0.5168
Early stopping:  0.1474472939959434
Epoch: 004, Loss: 2.4685, Train: 0.5671, Test: 0.5389
Early stopping:  0.20904086312435768
Epoch: 005, Loss: 2.2699, Train: 0.5899, Test: 0.5705
Early stopping:  0.27232267987651354
Epoch: 006, Loss: 2.0711, Train: 0.6104, Test: 0.5905
Early stopping:  0.29885855196152794
Epoch: 007, Loss: 1.8796, Train: 0.6309, Test: 0.6184
Early stopping:  0.3088029222987719
Epoch: 008, Loss: 1.7019, Train: 0.6404, Test: 0.6263
Early stopping:  0.30419721714024717
Epoch: 009, Loss: 1.5475, Train: 0.6449, Test: 0.6311
Early stopping:  0.2871609024585015
Epoch: 010, Loss: 1.4215, Train: 0.6512, Test: 0.6321
Early stopping:  0.25878232986934796
Epoch: 011, Loss: 1.3216, Train: 0.6562, Test: 0.6411
Early

In [ ]:
# Saving dataframes
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 515 ms (started: 2024-08-17 14:18:22 +00:00)
